In [2]:
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings('ignore')


Mounted at /content/drive


### 채권분석보고서(05~17년)

https://finance.naver.com/research/debenture_list.naver?&page=197

-신한

https://www.shinhansec.com/siw/wealth-management/bond-rp/bbs_gibond/view.do

#### import

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib import request

#### pdf 다운로드 크롤러

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib import request
import os

page = 1

comp_list=[]
date_list=[]
fail_list=[]

# 총 페이지 설정
total_page = 191

while True:
    url = 'https://finance.naver.com/research/debenture_list.nhn?' # 주소

    # 파라메터 선언
    params = {
        'page' : page
    }

    # 수프질
    resp = requests.get(url, params=params)
    soup = BeautifulSoup(resp.content, 'html.parser')

    # 링크덜
    td_link = soup.find('table', class_='type_1').find_all('td', class_='file')
    td_date = soup.find('table', class_='type_1').find_all('td', class_='date')
    tr_comp = soup.find('table', class_= 'type_1').find_all('tr')

    # 증권사 명 추출
    for tr in tr_comp:
        try:
            comp = tr.find_all('td')[1].text
            comp_list.append(comp)
            print(f'{comp}', end=' ')
        except:
            continue

    # 날짜 추출
    for idx, td in enumerate(td_date):
        if idx % 2 == 0:
            date_list.append(td.text)
            print(f'{td.text} 읽음', end=' ')

    # # 파일 다운로드.
    # for idx, td in enumerate(td_link):
    #     fname = f'{page}_{idx+1}.pdf'
    #     save_path = '/content/drive/Shareddrives/4조/data/bondreport/' + fname
    #     try:
    #       file_link = td.find('a')['href']
    #       request.urlretrieve(file_link, save_path)
    #       print(f'{comp_list[idx]} {date_list[idx]} {fname} 다운로드 완료')

    #     except:
    #       fail_list.append(fname)
    #       open(save_path, 'wb').close()  # 빈 PDF 파일 생성
    #       print(f'{comp_list[idx]} {date_list[idx]} {fname} 다운로드 실패')

    if page >= total_page: # 페이지 다 넘기면 탈출
        break

    page += 1 # 페이지 넘기기
    print()

print("다운로드 실패한 파일 목록:", fail_list)


#### pdf파일이 없는 리포트 리스트

In [ ]:
fail_list

['8_26.pdf',
 '10_22.pdf',
 '13_18.pdf',
 '15_3.pdf',
 '16_18.pdf',
 '18_30.pdf',
 '23_10.pdf',
 '26_3.pdf',
 '27_12.pdf',
 '57_25.pdf']

#### pdf처리를 위한 설치

In [ ]:
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.3 MB/s eta 0:00:00


#### 보고서 발행 회사 목록, 날짜 목록, pdf파일이 없는 리포트 리스트 저장/불러오기

In [ ]:
# import pickle

# with open("/content/drive/Shareddrives/4조/data/bondreport/comp_list.pkl","wb") as f:
#     pickle.dump(comp_list, f)
# with open("/content/drive/Shareddrives/4조/data/bondreport/date_list.pkl","wb") as f:
#     pickle.dump(date_list, f)
# with open("/content/drive/Shareddrives/4조/data/bondreport/fail_list.pkl","wb") as f:
#     pickle.dump(fail_list, f)

In [ ]:
import pickle

with open("/content/drive/Shareddrives/4조/data/bondreport/comp_list.pkl","rb") as f:
    comp_list = pickle.load(f)

with open("/content/drive/Shareddrives/4조/data/bondreport/date_list.pkl","rb") as f:
    date_list = pickle.load(f)

with open("/content/drive/Shareddrives/4조/data/bondreport/fail_list.pkl","rb") as f:
    fail_list = pickle.load(f)

#### pdf ==> text 변환 및 전처리

- V1
- V2 : 휴리스틱 접근: 테이블로 추정되는 텍스트 패턴을 찾아서 제거.(연속된 숫자와 구분 기호만 포함된 라인을 찾아서 제거)
- V3 : tabula

In [ ]:
# V1
def pdfToText(fname):

    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfpage import PDFPage
    from io import StringIO

    try:
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
        fp = open(f'/content/drive/Shareddrives/4조/data/bondreport/{fname}', 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ''
        maxpages = 0
        caching = True
        pagenos = set()

        for page in PDFPage.get_pages(fp, pagenos, maxpages = maxpages, password = password,
                                      caching = caching, check_extractable = True):
            interpreter.process_page(page)

        text = retstr.getvalue()

        fp.close()
        device.close()
        retstr.close()

        return text
    except Exception as e:
        return str(e)

In [ ]:
# V2
def pdfToText_v2(fname):
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfpage import PDFPage
    from io import StringIO
    import re

    try:
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
        fp = open(f'/content/drive/Shareddrives/4조/data/bondreport/{fname}', 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ''
        maxpages = 0
        caching = True
        pagenos = set()

        for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,
                                      caching=caching, check_extractable=True):
            interpreter.process_page(page)

        text = retstr.getvalue()

        # Remove lines that seem to be table rows (based on heuristic: lines with only numbers and some specific characters)
        lines = text.split('\n')
        cleaned_lines = [line for line in lines if not re.fullmatch(r"[0-9\s,.\-%$]+", line.strip())]
        cleaned_text = '\n'.join(cleaned_lines)

        fp.close()
        device.close()
        retstr.close()

        return cleaned_text
    except Exception as e:
        return str(e)

# As I cannot directly test the function here, you might want to try this updated function on your environment.


In [ ]:
# pdf에서 테이블 빼는 라이브러리
!pip install tabula-py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 45.6 MB/s eta 0:00:00


In [ ]:
#  V3 : tabula

def pdfToText_v3(fname):
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfpage import PDFPage
    from io import StringIO
    import tabula

    try:
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
        fp = open(f'/content/drive/Shareddrives/4조/data/bondreport/{fname}', 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ''
        maxpages = 0
        caching = True
        pagenos = set()

        for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,
                                      caching=caching, check_extractable=True):
            interpreter.process_page(page)

        text = retstr.getvalue()

        # tabula 사용해서 pdf 내 테이블 추출
        tables = tabula.read_pdf(f'/content/drive/Shareddrives/4조/data/bondreport/{fname}', pages='all', multiple_tables=True)

        # 테이블 텍스트로 변환해서 메인텍스트에서 제거
        for table in tables:
            table_text = table.to_string()
            text = text.replace(table_text, '')

        fp.close()
        device.close()
        retstr.close()

        return text
    except Exception as e:
        return str(e)

# The above function is a pseudo-code representation because I cannot directly run tabula in this environment.
# You can uncomment the tabula-related lines and run the function in your local environment after installing tabula-py.


#### V1, V2, V3 성능 비교

In [ ]:
abc=pdfToText('191_2.pdf')
abc

'Daewoo Bond Brief   \nDaewoo \n\n채권   윤일광(02-768-3058) ikyoon@bestez.com \n\n2008. 12. 23. Tue \n\n채권시황 \n\n(cid:122)  국고채 3년물 금리는 7bp 상승한 3.88%, 5년물 금리는 8bp 상승한 4.24%로 마감 \n\n(cid:122)  연말 북클로징에 들어간 가운데 전반적으로 거래 한산. 얇은 장세 속에 약간의 위험관리 매물이 출회되어도 금리가 밀리는 양상. \n\n(cid:122)  금리 반등의 가장 큰 이유는 당장 다음달부터 크게 늘어나게 되는 국고채 발행물량에 대한 부담. 이번 달에는 3.2조원 발행에 불과했으나 \n\n1월에는 거의 두 배인 6조원 수준으로 예상되며 이에 대한 우려가 일정부분 선반영되는 모습 \n\n(cid:122)  12월 금통위 이후 금리가 단기 급락한 가운데 연말 결산을 맞아 일종의 윈도우 드레싱 양상이 전개되고 있으나, 1월 수급 부담을 앞두고 \n일부 선제적 이익실현 욕구가 대두되면서 매물 출회. 환율이 반등하면서 외인들이 공격적인 국채선물 매도에 나선 것도 약세 조정에 일조 \n\n(cid:122)  한편, 그동안 outperform해 오던 공사채/은행채의 스프레드가 가격 메리트의 소진과 향후 잠재부실(은행채의 경우) 우려 등으로  더 이상 \n\n줄어들지 못하고 오히려 소폭 반등하려는 조짐  \n\n(cid:122)  주요 금리가 키 레벨에서 조정 받기 시작했고 스프레드 역시 추가 축소에 부담을 느끼는 가운데 연말 조심스러운 관리 모드에 진입한 양\n\n상. 큰 폭의 금리 변동 가능성은 높지 않으나, 전반적으로 기간 조정 내지 약간의 약보합(즉 소폭의 금리 반등) 나타날 전망 \n\n관심 차트 및 경제지표 \n\n단기 급락에 따른 피로감에 기간 조정 양상  \n\n  대규모 국채 입찰에 따른 물량 압박으로 금리 상승 \n\n기준금리\n국고채(3년)\n국고채(10년)\n\nCD(91일)\n통안채(1년)\n회사채(무보증3년)AA-

In [ ]:
abc=pdfToText_v2('191_2.pdf')
abc

'Daewoo Bond Brief   \nDaewoo \n\n채권   윤일광(02-768-3058) ikyoon@bestez.com \n\n2008. 12. 23. Tue \n\n채권시황 \n\n(cid:122)  국고채 3년물 금리는 7bp 상승한 3.88%, 5년물 금리는 8bp 상승한 4.24%로 마감 \n\n(cid:122)  연말 북클로징에 들어간 가운데 전반적으로 거래 한산. 얇은 장세 속에 약간의 위험관리 매물이 출회되어도 금리가 밀리는 양상. \n\n(cid:122)  금리 반등의 가장 큰 이유는 당장 다음달부터 크게 늘어나게 되는 국고채 발행물량에 대한 부담. 이번 달에는 3.2조원 발행에 불과했으나 \n\n1월에는 거의 두 배인 6조원 수준으로 예상되며 이에 대한 우려가 일정부분 선반영되는 모습 \n\n(cid:122)  12월 금통위 이후 금리가 단기 급락한 가운데 연말 결산을 맞아 일종의 윈도우 드레싱 양상이 전개되고 있으나, 1월 수급 부담을 앞두고 \n일부 선제적 이익실현 욕구가 대두되면서 매물 출회. 환율이 반등하면서 외인들이 공격적인 국채선물 매도에 나선 것도 약세 조정에 일조 \n\n(cid:122)  한편, 그동안 outperform해 오던 공사채/은행채의 스프레드가 가격 메리트의 소진과 향후 잠재부실(은행채의 경우) 우려 등으로  더 이상 \n\n줄어들지 못하고 오히려 소폭 반등하려는 조짐  \n\n(cid:122)  주요 금리가 키 레벨에서 조정 받기 시작했고 스프레드 역시 추가 축소에 부담을 느끼는 가운데 연말 조심스러운 관리 모드에 진입한 양\n\n상. 큰 폭의 금리 변동 가능성은 높지 않으나, 전반적으로 기간 조정 내지 약간의 약보합(즉 소폭의 금리 반등) 나타날 전망 \n\n관심 차트 및 경제지표 \n\n단기 급락에 따른 피로감에 기간 조정 양상  \n\n  대규모 국채 입찰에 따른 물량 압박으로 금리 상승 \n\n기준금리\n국고채(3년)\n국고채(10년)\n\nCD(91일)\n통안채(1년)\n회사채(무보증3년)AA-

In [ ]:
abc=pdfToText_v3('191_2.pdf')
abc

'Daewoo Bond Brief   \nDaewoo \n\n채권   윤일광(02-768-3058) ikyoon@bestez.com \n\n2008. 12. 23. Tue \n\n채권시황 \n\n(cid:122)  국고채 3년물 금리는 7bp 상승한 3.88%, 5년물 금리는 8bp 상승한 4.24%로 마감 \n\n(cid:122)  연말 북클로징에 들어간 가운데 전반적으로 거래 한산. 얇은 장세 속에 약간의 위험관리 매물이 출회되어도 금리가 밀리는 양상. \n\n(cid:122)  금리 반등의 가장 큰 이유는 당장 다음달부터 크게 늘어나게 되는 국고채 발행물량에 대한 부담. 이번 달에는 3.2조원 발행에 불과했으나 \n\n1월에는 거의 두 배인 6조원 수준으로 예상되며 이에 대한 우려가 일정부분 선반영되는 모습 \n\n(cid:122)  12월 금통위 이후 금리가 단기 급락한 가운데 연말 결산을 맞아 일종의 윈도우 드레싱 양상이 전개되고 있으나, 1월 수급 부담을 앞두고 \n일부 선제적 이익실현 욕구가 대두되면서 매물 출회. 환율이 반등하면서 외인들이 공격적인 국채선물 매도에 나선 것도 약세 조정에 일조 \n\n(cid:122)  한편, 그동안 outperform해 오던 공사채/은행채의 스프레드가 가격 메리트의 소진과 향후 잠재부실(은행채의 경우) 우려 등으로  더 이상 \n\n줄어들지 못하고 오히려 소폭 반등하려는 조짐  \n\n(cid:122)  주요 금리가 키 레벨에서 조정 받기 시작했고 스프레드 역시 추가 축소에 부담을 느끼는 가운데 연말 조심스러운 관리 모드에 진입한 양\n\n상. 큰 폭의 금리 변동 가능성은 높지 않으나, 전반적으로 기간 조정 내지 약간의 약보합(즉 소폭의 금리 반등) 나타날 전망 \n\n관심 차트 및 경제지표 \n\n단기 급락에 따른 피로감에 기간 조정 양상  \n\n  대규모 국채 입찰에 따른 물량 압박으로 금리 상승 \n\n기준금리\n국고채(3년)\n국고채(10년)\n\nCD(91일)\n통안채(1년)\n회사채(무보증3년)AA-

In [ ]:
type(abc)

str

#### 전체 데이터 전처리하여 csv파일로 저장

##### 테스트

In [ ]:
# import pandas as pd

# text_list=[]
# for idx, comp in enumerate(comp_list[:60]):

#     page = (idx // 30)+1
#     p_no = (idx % 30)+1

#     text=pdfToText(f'{page}_{p_no}.pdf')

#     print(f'{page}_{p_no}.pdf 텍스트 추출...')
#     text=text.replace('\n','')
#     text=text.replace('\r','')
#     text=text.replace('\t','')

#     text_list.append(text)

#     if (idx+1) % 30 == 0:
#         dict_report ={
#             '증권사' : comp_list[idx-29:idx+1],
#             '날짜' : date_list[idx-29:idx+1],
#             '텍스트' : text_list
#         }

#         df = pd.DataFrame(dict_report)
#         df.to_csv(f'bond_report page_{page}.csv', index=False)

#         text_list = []


# rest = len(text_list)

# dict_report ={
#             '증권사' : comp_list[len(comp_list)-rest:],
#             '날짜' : date_list[len(date_list)-rest:],
#             '텍스트' : text_list
#             }

# df = pd.DataFrame(dict_report)
# df.to_csv(f'/content/drive/Shareddrives/4조/data/bondreport/final/page_{page}.csv', index=False)

1_1.pdf 텍스트 추출...
1_2.pdf 텍스트 추출...
1_3.pdf 텍스트 추출...
1_4.pdf 텍스트 추출...
1_5.pdf 텍스트 추출...
1_6.pdf 텍스트 추출...
1_7.pdf 텍스트 추출...
1_8.pdf 텍스트 추출...
1_9.pdf 텍스트 추출...
1_10.pdf 텍스트 추출...
1_11.pdf 텍스트 추출...
1_12.pdf 텍스트 추출...
1_13.pdf 텍스트 추출...
1_14.pdf 텍스트 추출...
1_15.pdf 텍스트 추출...
1_16.pdf 텍스트 추출...
1_17.pdf 텍스트 추출...
1_18.pdf 텍스트 추출...
1_19.pdf 텍스트 추출...
1_20.pdf 텍스트 추출...
1_21.pdf 텍스트 추출...
1_22.pdf 텍스트 추출...
1_23.pdf 텍스트 추출...
1_24.pdf 텍스트 추출...
1_25.pdf 텍스트 추출...
1_26.pdf 텍스트 추출...
1_27.pdf 텍스트 추출...
1_28.pdf 텍스트 추출...
1_29.pdf 텍스트 추출...
1_30.pdf 텍스트 추출...
2_1.pdf 텍스트 추출...
2_2.pdf 텍스트 추출...
2_3.pdf 텍스트 추출...
2_4.pdf 텍스트 추출...
2_5.pdf 텍스트 추출...
2_6.pdf 텍스트 추출...
2_7.pdf 텍스트 추출...
2_8.pdf 텍스트 추출...
2_9.pdf 텍스트 추출...
2_10.pdf 텍스트 추출...
2_11.pdf 텍스트 추출...
2_12.pdf 텍스트 추출...
2_13.pdf 텍스트 추출...
2_14.pdf 텍스트 추출...
2_15.pdf 텍스트 추출...
2_16.pdf 텍스트 추출...
2_17.pdf 텍스트 추출...
2_18.pdf 텍스트 추출...
2_19.pdf 텍스트 추출...
2_20.pdf 텍스트 추출...
2_21.pdf 텍스트 추출...
2_22.pdf 텍스트 추출...
2_23.pdf 텍스트 추출...
2_24.pdf 텍스

##### V2를 적용한 전체 파일 csv화

- 빈 pdf를 생성한 부분이 있어 해당 부분 리스트로 체크하여 제거 필요

- 테이블을 깔끔하게 처리하지 못하고 칼럼명이 살아있는 이슈

- Compliance Notice 부분 삭제 방안 필.
(Compliance Notice 뒤의 텍스트는 자른다
본 자료는 /대우/신한
이 자료는 /한화)

In [ ]:
# Revised code
def save_to_csv():
    # Assuming comp_list, date_list and pdfToText function are defined elsewhere

    text_list = []
    for idx, comp in enumerate(comp_list[1080:]):

        page = (idx // 30) + 1 +36
        p_no = (idx % 30) + 1

        text = pdfToText_v2(f'{page}_{p_no}.pdf')

        print(f'{page}_{p_no}.pdf 텍스트 추출...')
        text = text.replace('\n', '')
        text = text.replace('\r', '')
        text = text.replace('\t', '')

        text_list.append(text)

        if (idx + 1) % 30 == 0 or idx == len(comp_list) - 1:  # added a condition to handle the last batch
            start_idx = idx - 29 if idx + 1 >= 30 else 0  # handle the first batch

            dict_report = {
                '증권사': comp_list[start_idx:idx + 1],
                '날짜': date_list[start_idx:idx + 1],
                '텍스트': text_list
            }

            df = pd.DataFrame(dict_report)
            if idx == len(comp_list) - 1:  # handle the last batch
                df.to_csv(f'/content/drive/Shareddrives/4조/data/bondreport/final/page_{page}.csv', index=False)
            else:
                df.to_csv(f'/content/drive/Shareddrives/4조/data/bondreport/final/bond_report page_{page}.csv', index=False)

            text_list = []

#Run
save_to_csv()


1_1.pdf 텍스트 추출...
1_2.pdf 텍스트 추출...
1_3.pdf 텍스트 추출...
1_4.pdf 텍스트 추출...
1_5.pdf 텍스트 추출...
1_6.pdf 텍스트 추출...
1_7.pdf 텍스트 추출...
1_8.pdf 텍스트 추출...
1_9.pdf 텍스트 추출...
1_10.pdf 텍스트 추출...
1_11.pdf 텍스트 추출...
1_12.pdf 텍스트 추출...
1_13.pdf 텍스트 추출...
1_14.pdf 텍스트 추출...
1_15.pdf 텍스트 추출...
1_16.pdf 텍스트 추출...
1_17.pdf 텍스트 추출...
1_18.pdf 텍스트 추출...
1_19.pdf 텍스트 추출...
1_20.pdf 텍스트 추출...
1_21.pdf 텍스트 추출...
1_22.pdf 텍스트 추출...
1_23.pdf 텍스트 추출...
1_24.pdf 텍스트 추출...
1_25.pdf 텍스트 추출...
1_26.pdf 텍스트 추출...
1_27.pdf 텍스트 추출...
1_28.pdf 텍스트 추출...
1_29.pdf 텍스트 추출...
1_30.pdf 텍스트 추출...
2_1.pdf 텍스트 추출...
2_2.pdf 텍스트 추출...
2_3.pdf 텍스트 추출...
2_4.pdf 텍스트 추출...
2_5.pdf 텍스트 추출...
2_6.pdf 텍스트 추출...
2_7.pdf 텍스트 추출...
2_8.pdf 텍스트 추출...
2_9.pdf 텍스트 추출...
2_10.pdf 텍스트 추출...
2_11.pdf 텍스트 추출...
2_12.pdf 텍스트 추출...
2_13.pdf 텍스트 추출...
2_14.pdf 텍스트 추출...
2_15.pdf 텍스트 추출...
2_16.pdf 텍스트 추출...
2_17.pdf 텍스트 추출...
2_18.pdf 텍스트 추출...
2_19.pdf 텍스트 추출...
2_20.pdf 텍스트 추출...
2_21.pdf 텍스트 추출...
2_22.pdf 텍스트 추출...
2_23.pdf 텍스트 추출...
2_24.pdf 텍스

OSError: ignored

#### 데이터 합치기

In [ ]:
import pandas as pd
page=1
df = pd.read_csv(f'/content/drive/Shareddrives/4조/data/bondreport/final/bond_report page_{page}.csv', index_col=0)
for i in range(2, 192):
  page=i
  df2 = pd.read_csv(f'/content/drive/Shareddrives/4조/data/bondreport/final/bond_report page_{page}.csv', index_col=0)
  df = pd.concat([df,df2])


In [ ]:
df

,날짜,텍스트
증권사,,
교보증권,23.08.30,Daily 2023.08.30 Fixed Income 백윤민 02_3771_91...
하나증권,23.08.30,2023년 8월 30일 I Global Asset Research Credit...
유진투자증권,23.08.30,Eugene’s FICC Update Fixed Income 김지나_ 02)368-...
유안타증권,23.08.30,"(단위: 억원, %)8.29 (화)1d5d1M전일 주요 채권 관련 기사CP 91dC..."
다올투자증권,23.08.30,"30 Aug 202330y91d(%, )(%, )AAAAAAAAAAAAAAAAAAA..."
...,...,...
현대차증권,12.09.28,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...
키움증권,12.09.24,채권메모 - 2008. 11. 26. 수 서철수 02-768-3501 cybersc...
KTB투자증권,12.09.24,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...


In [ ]:
df.to_csv(f'/content/drive/Shareddrives/4조/data/bondreport/final/bond_report_concat.csv', index=False)

In [ ]:
df['텍스트'].unique

<bound method Series.unique of 교보증권        Daily 2023.08.30 Fixed Income 백윤민 02_3771_91...
하나증권      2023년  8월  30일  I Global Asset Research Credit...
유진투자증권    Eugene’s FICC Update Fixed Income 김지나_ 02)368-...
유안타증권     (단위: 억원, %)8.29 (화)1d5d1M전일 주요 채권 관련 기사CP 91dC...
다올투자증권    30 Aug 202330y91d(%, )(%, )AAAAAAAAAAAAAAAAAAA...
                                ...                        
대우증권      Daewoo Bond Brief   Daewoo 채권   윤일광(02-768-305...
대우증권      채권메모 - 2008. 11. 26. 수 서철수 02-768-3501 cybersc...
대우증권      Daewoo Bond Brief   Daewoo 채권   윤일광(02-768-305...
대우증권      Daewoo Bond Brief   Daewoo 채권   윤일광(02-768-305...
대우증권      채권메모 - 2008. 11. 24. 월 서철수 02-768-3501 cybersc...
Name: 텍스트, Length: 5730, dtype: object>

In [ ]:
fail_list

['8_26.pdf',
 '10_22.pdf',
 '13_18.pdf',
 '15_3.pdf',
 '16_18.pdf',
 '18_30.pdf',
 '23_10.pdf',
 '26_3.pdf',
 '27_12.pdf',
 '57_25.pdf']

In [ ]:
len(comp_list), len(date_list)

(5730, 5730)

In [ ]:
len(df)

5730

In [ ]:
df.columns

Index(['날짜', '텍스트'], dtype='object')

In [ ]:
df.index

Index(['교보증권', '하나증권', '유진투자증권', '유안타증권', '다올투자증권', '키움증권', '유안타증권', '한화투자증권',
       '교보증권', '다올투자증권',
       ...
       '대우증권', '대우증권', '대우증권', '대우증권', '대우증권', '대우증권', '대우증권', '대우증권', '대우증권',
       '대우증권'],
      dtype='object', length=5730)

In [ ]:
df.index=comp_list

In [ ]:
df['날짜']=date_list

In [ ]:
df

,날짜,텍스트
교보증권,23.08.30,Daily 2023.08.30 Fixed Income 백윤민 02_3771_91...
하나증권,23.08.30,2023년 8월 30일 I Global Asset Research Credit...
유진투자증권,23.08.30,Eugene’s FICC Update Fixed Income 김지나_ 02)368-...
유안타증권,23.08.30,"(단위: 억원, %)8.29 (화)1d5d1M전일 주요 채권 관련 기사CP 91dC..."
다올투자증권,23.08.30,"30 Aug 202330y91d(%, )(%, )AAAAAAAAAAAAAAAAAAA..."
...,...,...
대우증권,08.11.27,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...
대우증권,08.11.26,채권메모 - 2008. 11. 26. 수 서철수 02-768-3501 cybersc...
대우증권,08.11.26,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...
대우증권,08.11.25,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...


In [ ]:
df.to_csv(f'/content/drive/Shareddrives/4조/data/bondreport/final/bond_report_concat.csv')

In [ ]:
df.dropna()

,날짜,텍스트
교보증권,23.08.30,Daily 2023.08.30 Fixed Income 백윤민 02_3771_91...
하나증권,23.08.30,2023년 8월 30일 I Global Asset Research Credit...
유진투자증권,23.08.30,Eugene’s FICC Update Fixed Income 김지나_ 02)368-...
유안타증권,23.08.30,"(단위: 억원, %)8.29 (화)1d5d1M전일 주요 채권 관련 기사CP 91dC..."
다올투자증권,23.08.30,"30 Aug 202330y91d(%, )(%, )AAAAAAAAAAAAAAAAAAA..."
...,...,...
대우증권,08.11.27,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...
대우증권,08.11.26,채권메모 - 2008. 11. 26. 수 서철수 02-768-3501 cybersc...
대우증권,08.11.26,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...
대우증권,08.11.25,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...


#### 데이터 살펴보기

In [14]:
import pandas as pd
import numpy as np

In [20]:
df = pd.read_csv('/content/drive/Shareddrives/4조/data/bondreport/final/bond_report_concat.csv')
df

,Unnamed: 0,날짜,텍스트
0,교보증권,23.08.30,Daily 2023.08.30 Fixed Income 백윤민 02_3771_91...
1,하나증권,23.08.30,2023년 8월 30일 I Global Asset Research Credit...
2,유진투자증권,23.08.30,Eugene’s FICC Update Fixed Income 김지나_ 02)368-...
3,유안타증권,23.08.30,"(단위: 억원, %)8.29 (화)1d5d1M전일 주요 채권 관련 기사CP 91dC..."
4,다올투자증권,23.08.30,"30 Aug 202330y91d(%, )(%, )AAAAAAAAAAAAAAAAAAA..."
...,...,...,...
5725,대우증권,08.11.27,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...
5726,대우증권,08.11.26,채권메모 - 2008. 11. 26. 수 서철수 02-768-3501 cybersc...
5727,대우증권,08.11.26,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...
5728,대우증권,08.11.25,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...


In [21]:
df.columns = ['증권사', '날짜', '텍스트']
df

,증권사,날짜,텍스트
0,교보증권,23.08.30,Daily 2023.08.30 Fixed Income 백윤민 02_3771_91...
1,하나증권,23.08.30,2023년 8월 30일 I Global Asset Research Credit...
2,유진투자증권,23.08.30,Eugene’s FICC Update Fixed Income 김지나_ 02)368-...
3,유안타증권,23.08.30,"(단위: 억원, %)8.29 (화)1d5d1M전일 주요 채권 관련 기사CP 91dC..."
4,다올투자증권,23.08.30,"30 Aug 202330y91d(%, )(%, )AAAAAAAAAAAAAAAAAAA..."
...,...,...,...
5725,대우증권,08.11.27,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...
5726,대우증권,08.11.26,채권메모 - 2008. 11. 26. 수 서철수 02-768-3501 cybersc...
5727,대우증권,08.11.26,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...
5728,대우증권,08.11.25,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...


In [22]:
df.describe()

,증권사,날짜,텍스트
count,5730,5730,5730
unique,25,2509,5611
top,하나증권,23.03.13,Fixed Income Report Weekly
freq,989,15,81


In [23]:
df['증권사'].value_counts().head(10)

하나증권        989
현대차증권       613
IBK투자증권     552
이베스트투자증권    424
대우증권        366
키움증권        336
신한투자증권      309
미래에셋증권      308
교보증권        289
유안타증권       279
Name: 증권사, dtype: int64

In [24]:
df['증권사'].value_counts(normalize=True)

하나증권        0.172600
현대차증권       0.106981
IBK투자증권     0.096335
이베스트투자증권    0.073997
대우증권        0.063874
키움증권        0.058639
신한투자증권      0.053927
미래에셋증권      0.053752
교보증권        0.050436
유안타증권       0.048691
이베스트증권      0.041187
KTB투자증권     0.032461
유진투자증권      0.031937
다올투자증권      0.028621
한화투자증권      0.027749
메리츠종금증권     0.016056
이트레이드증권     0.008551
하이투자증권      0.008202
LIG투자증권     0.007330
대신증권        0.006806
DS투자증권      0.003665
메리츠증권       0.003316
케이프투자증권     0.002269
한국투자증권      0.001920
DB금융투자      0.000698
Name: 증권사, dtype: float64

#### 증권사별 전처리

In [25]:
checklist = df['증권사'].unique()
checklist=checklist.tolist()

##### 공통

In [140]:
df['텍스트']=df.텍스트.str.split('본 자료는').str[0]
df['텍스트']=df.텍스트.str.split('본  자료는').str[0]
df['텍스트']=df.텍스트.str.split('동 자료는').str[0]
df['텍스트']=df.텍스트.str.split('동  자료는').str[0]

df['텍스트']=df.텍스트.str.split('본 자료에').str[0]
df['텍스트']=df.텍스트.str.split('본  자료에').str[0]
df['텍스트']=df.텍스트.str.split('동 자료에').str[0]
df['텍스트']=df.텍스트.str.split('동  자료에').str[0]
df['텍스트']=df.텍스트.str.split('본  조사자료').str[0]
df['텍스트']=df.텍스트.str.split('본  조사분석자료').str[0]




In [ ]:
# 대괄호 안에 문자 지우기
def remove_brackets_content1(text):
    pattern = r'\[.*?\]'
    return re.sub(pattern, '', text)

def remove_brackets_content2(text):
    pattern = r'\(.*?\)'
    return re.sub(pattern, '', text)
def remove_brackets_content3(text):
    pattern = r'\<.*?\>'
    return re.sub(pattern, '', text)

# Apply the function to '텍스트' column
df['텍스트'] = df['텍스트'].apply(remove_brackets_content1)
df['텍스트'] = df['텍스트'].apply(remove_brackets_content2)
df['텍스트'] = df['텍스트'].apply(remove_brackets_content3)

##### 다올투자증권

##### 다올투자증권은 글자가 포함되어 있지 않아 삭제

In [26]:
# 수치만 보여주는 리포트
df.drop(df[df['증권사']=='다올투자증권'].index, inplace=True)
df

,증권사,날짜,텍스트
0,교보증권,23.08.30,Daily 2023.08.30 Fixed Income 백윤민 02_3771_91...
1,하나증권,23.08.30,2023년 8월 30일 I Global Asset Research Credit...
2,유진투자증권,23.08.30,Eugene’s FICC Update Fixed Income 김지나_ 02)368-...
3,유안타증권,23.08.30,"(단위: 억원, %)8.29 (화)1d5d1M전일 주요 채권 관련 기사CP 91dC..."
5,키움증권,23.08.30,2023 년 08 월 30 일 키움증권 리서치센터 투자전략팀 채권전략 안예하 yha...
...,...,...,...
5725,대우증권,08.11.27,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...
5726,대우증권,08.11.26,채권메모 - 2008. 11. 26. 수 서철수 02-768-3501 cybersc...
5727,대우증권,08.11.26,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...
5728,대우증권,08.11.25,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...


In [28]:
checklist.remove('다올투자증권')
checklist

['교보증권',
 '하나증권',
 '유진투자증권',
 '유안타증권',
 '키움증권',
 '한화투자증권',
 '이베스트증권',
 '신한투자증권',
 '메리츠증권',
 '하이투자증권',
 'DS투자증권',
 '한국투자증권',
 '대신증권',
 '미래에셋증권',
 '현대차증권',
 'IBK투자증권',
 'DB금융투자',
 '케이프투자증권',
 '이베스트투자증권',
 'LIG투자증권',
 'KTB투자증권',
 '이트레이드증권',
 '메리츠종금증권',
 '대우증권']

##### 한국투자증권

In [30]:
# df[df['증권사']=='한국투자증권'].iloc[0,2]
df[df['증권사']=='한국투자증권']

,증권사,날짜,텍스트
235,한국투자증권,23.07.24,No /Root object! - Is this really a PDF?
291,한국투자증권,23.07.13,No /Root object! - Is this really a PDF?
377,한국투자증권,23.06.29,No /Root object! - Is this really a PDF?
422,한국투자증권,23.06.21,No /Root object! - Is this really a PDF?
467,한국투자증권,23.06.13,No /Root object! - Is this really a PDF?
539,한국투자증권,23.06.01,No /Root object! - Is this really a PDF?
669,한국투자증권,23.05.11,No /Root object! - Is this really a PDF?
752,한국투자증권,23.04.25,No /Root object! - Is this really a PDF?
791,한국투자증권,23.04.19,No /Root object! - Is this really a PDF?
2574,한국투자증권,18.09.28,"차이나 Note 중국A주, 글로벌지수 편입 가속화 MSCI, 신흥시장..."


##### "No /Root object! - Is this really a PDF?" 삭제

In [31]:
df.drop(df[df['텍스트']=="No /Root object! - Is this really a PDF?"].index, inplace=True)
df

,증권사,날짜,텍스트
0,교보증권,23.08.30,Daily 2023.08.30 Fixed Income 백윤민 02_3771_91...
1,하나증권,23.08.30,2023년 8월 30일 I Global Asset Research Credit...
2,유진투자증권,23.08.30,Eugene’s FICC Update Fixed Income 김지나_ 02)368-...
3,유안타증권,23.08.30,"(단위: 억원, %)8.29 (화)1d5d1M전일 주요 채권 관련 기사CP 91dC..."
5,키움증권,23.08.30,2023 년 08 월 30 일 키움증권 리서치센터 투자전략팀 채권전략 안예하 yha...
...,...,...,...
5725,대우증권,08.11.27,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...
5726,대우증권,08.11.26,채권메모 - 2008. 11. 26. 수 서철수 02-768-3501 cybersc...
5727,대우증권,08.11.26,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...
5728,대우증권,08.11.25,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...


In [62]:
df[df['증권사']=='한국투자증권'].iloc[0,2]

'차이나   Note    중국A주,  글로벌지수  편입  가속화 MSCI, 신흥시장(EM) 지수에 중국A주 추가편입 계획 로드맵 발표  2019년 2월 말~3월초 최종안 발표, 편입되면 660억달러 신규자금 유입 A주의 FTSE 편입도 확정, 가속화되는 해외자금 유입에 대형 우량주 수혜 편입비중을     5%5%5%5%에서에서에서에서     20%20%20%20%로로로로     상향상향상향상향     계획계획계획계획     발표발표발표발표 MSCI,  중국중국중국중국AAAA주의주의주의주의     편입비중을MSCI, 편입비중을편입비중을MSCI, MSCI, 지난  25일  MSCI는  중국A주의  신흥시장  지수(이하  EM지수)  추가  편입에  대한 가이드라인을  발표했다.  구체적인  내용은  세 가지였다.  올해  5월과 8월  편입된  중국A주 대형주 비중을 기존 5%에서 20%까지 늘리겠다는 것과, 투자 가능한 중국 주식 유니버스를 차이넥스트 지수까지 확대한다는 것, 그리고 2020년 5월에는 중국 A주 중형주 중 20%를 편입하겠다는 것이다.  MSCI의  가이드라인  대로 편입이 진행될  경우  현재 0.71%의  비중으로  편입된  중국A주는  내년  8월  2.8%,  내후년  5월에는  3.4%까지  비중이  확대될  전망이다. MSCI는  중국  증시의  완전  편입을  가정할  경우  중국  전체  비중은  약  40%가  될 전망이며, 그중 A주가 차지하는 비중은 약 14.8% 정도가 될 것으로 예상했다.  MSCI는 투자업계의 피드백을 2019년 2월 15일까지 반영하고, 28일 전후로 최종안을 발표할 계획이다. 따라서 최종안은 현재 MSCI에서 제시한 안과 다를 가능성도  충분하다.  MSCI  측에서  예상하는  주요  쟁점은  1)  20%인  중국  A주  대형주와 중형주  편입  비중의  적정성,  2)  중국  A주의  추가  편입과  사우디아라비아  및  아르헨티나 지수의 EM 지수 편입 간 타이밍, 3) 중국 A주 중형주

In [70]:
import pandas as pd
import re

# 같은 단어가 4번 반복되는 패턴을 찾아서 지우기
def remove_except_one(text):
    pattern = r'(\w+)(\1{3,})'
    return re.sub(pattern, r'\1', text)

# Apply function to '텍스트' column
df['텍스트'] = df['텍스트'].apply(remove_except_one)

In [71]:
df[df['증권사']=='한국투자증권'].iloc[0,2]

'차이나   Note    중국A주,  글로벌지수  편입  가속화 MSCI, 신흥시장(EM) 지수에 중국A주 추가편입 계획 로드맵 발표  2019년 2월 말~3월초 최종안 발표, 편입되면 660억달러 신규자금 유입 A주의 FTSE 편입도 확정, 가속화되는 해외자금 유입에 대형 우량주 수혜 편입비중을     5%5%5%5%에서     20%20%20%20%로     상향     계획     발표 MSCI,  중국A주의     편입비중을MSCI, 편입비중을편입비중을MSCI, MSCI, 지난  25일  MSCI는  중국A주의  신흥시장  지수(이하  EM지수)  추가  편입에  대한 가이드라인을  발표했다.  구체적인  내용은  세 가지였다.  올해  5월과 8월  편입된  중국A주 대형주 비중을 기존 5%에서 20%까지 늘리겠다는 것과, 투자 가능한 중국 주식 유니버스를 차이넥스트 지수까지 확대한다는 것, 그리고 2020년 5월에는 중국 A주 중형주 중 20%를 편입하겠다는 것이다.  MSCI의  가이드라인  대로 편입이 진행될  경우  현재 0.71%의  비중으로  편입된  중국A주는  내년  8월  2.8%,  내후년  5월에는  3.4%까지  비중이  확대될  전망이다. MSCI는  중국  증시의  완전  편입을  가정할  경우  중국  전체  비중은  약  40%가  될 전망이며, 그중 A주가 차지하는 비중은 약 14.8% 정도가 될 것으로 예상했다.  MSCI는 투자업계의 피드백을 2019년 2월 15일까지 반영하고, 28일 전후로 최종안을 발표할 계획이다. 따라서 최종안은 현재 MSCI에서 제시한 안과 다를 가능성도  충분하다.  MSCI  측에서  예상하는  주요  쟁점은  1)  20%인  중국  A주  대형주와 중형주  편입  비중의  적정성,  2)  중국  A주의  추가  편입과  사우디아라비아  및  아르헨티나 지수의 EM 지수 편입 간 타이밍, 3) 중국 A주 중형주 168종목을 모두 커버하는데 소요될 시간, 4) 차이넥스트 지수의 편입 가

In [76]:
# # 대괄호 안에 문자 지우기
# def remove_brackets_content1(text):
#     pattern = r'\[.*?\]'
#     return re.sub(pattern, '', text)

# def remove_brackets_content2(text):
#     pattern = r'\(.*?\)'
#     return re.sub(pattern, '', text)
# def remove_brackets_content3(text):
#     pattern = r'\<.*?\>'
#     return re.sub(pattern, '', text)

# # Apply the function to '텍스트' column
# df['텍스트'] = df['텍스트'].apply(remove_brackets_content1)
# df['텍스트'] = df['텍스트'].apply(remove_brackets_content2)
# df['텍스트'] = df['텍스트'].apply(remove_brackets_content3)

In [77]:
df[df['증권사']=='한국투자증권'].iloc[0,2]

'차이나   Note    중국A주,  글로벌지수  편입  가속화 MSCI, 신흥시장 지수에 중국A주 추가편입 계획 로드맵 발표  2019년 2월 말~3월초 최종안 발표, 편입되면 660억달러 신규자금 유입 A주의 FTSE 편입도 확정, 가속화되는 해외자금 유입에 대형 우량주 수혜 편입비중을     5%5%5%5%에서     20%20%20%20%로     상향     계획     발표 MSCI,  중국A주의     편입비중을MSCI, 편입비중을편입비중을MSCI, MSCI, 지난  25일  MSCI는  중국A주의  신흥시장  지수  추가  편입에  대한 가이드라인을  발표했다.  구체적인  내용은  세 가지였다.  올해  5월과 8월  편입된  중국A주 대형주 비중을 기존 5%에서 20%까지 늘리겠다는 것과, 투자 가능한 중국 주식 유니버스를 차이넥스트 지수까지 확대한다는 것, 그리고 2020년 5월에는 중국 A주 중형주 중 20%를 편입하겠다는 것이다.  MSCI의  가이드라인  대로 편입이 진행될  경우  현재 0.71%의  비중으로  편입된  중국A주는  내년  8월  2.8%,  내후년  5월에는  3.4%까지  비중이  확대될  전망이다. MSCI는  중국  증시의  완전  편입을  가정할  경우  중국  전체  비중은  약  40%가  될 전망이며, 그중 A주가 차지하는 비중은 약 14.8% 정도가 될 것으로 예상했다.  MSCI는 투자업계의 피드백을 2019년 2월 15일까지 반영하고, 28일 전후로 최종안을 발표할 계획이다. 따라서 최종안은 현재 MSCI에서 제시한 안과 다를 가능성도  충분하다.  MSCI  측에서  예상하는  주요  쟁점은  1)  20%인  중국  A주  대형주와 중형주  편입  비중의  적정성,  2)  중국  A주의  추가  편입과  사우디아라비아  및  아르헨티나 지수의 EM 지수 편입 간 타이밍, 3) 중국 A주 중형주 168종목을 모두 커버하는데 소요될 시간, 4) 차이넥스트 지수의 편입 가능 목록 포함 등이다.  

In [33]:
checklist.remove('한국투자증권')
checklist

['교보증권',
 '하나증권',
 '유진투자증권',
 '유안타증권',
 '키움증권',
 '한화투자증권',
 '이베스트증권',
 '신한투자증권',
 '메리츠증권',
 '하이투자증권',
 'DS투자증권',
 '대신증권',
 '미래에셋증권',
 '현대차증권',
 'IBK투자증권',
 'DB금융투자',
 '케이프투자증권',
 '이베스트투자증권',
 'LIG투자증권',
 'KTB투자증권',
 '이트레이드증권',
 '메리츠종금증권',
 '대우증권']

##### 유안타일반 리포트는 좋으나, 데일리 리포트는 전일 기사를 요약해서 보여주는 것 외에 텍스트가 존재하지 않음.

In [34]:
df[df['증권사']=='유안타증권']

,증권사,날짜,텍스트
3,유안타증권,23.08.30,"(단위: 억원, %)8.29 (화)1d5d1M전일 주요 채권 관련 기사CP 91dC..."
6,유안타증권,23.08.29,"(단위: 억원, %)8.28 (월)1d5d1M전일 주요 채권 관련 기사CP 91dC..."
18,유안타증권,23.08.28,"(단위: 억원, %)8.25 (금)1d5d1M전일 주요 채권 관련 기사CP 91dC..."
26,유안타증권,23.08.25,"(단위: 억원, %)8.24 (목)1d5d1M전일 주요 채권 관련 기사CP 91dC..."
41,유안타증권,23.08.24,"(단위: 억원, %)8.23 (수)1d5d1M전일 주요 채권 관련 기사CP 91dC..."
...,...,...,...
1922,유안타증권,21.07.22,"(단위: 억원, %)7.21 (수)1d5d1M전일 주요 채권 관련 기사CP 91dC..."
1924,유안타증권,21.07.21,"(단위: 억원, %)7.20 (화)1d5d1M전일 주요 채권 관련 기사CP 91dC..."
1926,유안타증권,21.07.19,"(단위: 억원, %)7.16 (금)1d5d1M전일 주요 채권 관련 기사CP 91dC..."
1928,유안타증권,21.07.16,"(단위: 억원, %)7.15 (목)1d5d1M전일 주요 채권 관련 기사CP 91dC..."


In [35]:
# 유안타 데일리 리포트는 전일 주요 채권 관련 기사를 요약한 것을 보여주고 있고 상황에 대한 분석은 포함되어 있지 않음.
df[df['증권사']=='유안타증권'].iloc[1,2]

'(단위: 억원, %)8.28 (월)1d5d1M전일 주요 채권 관련 기사CP 91dCD 91dKTB 3yKTB 5y+0.0 +0.0 +0.0 KTB 10yKTB 20yKTB 30y국민주택 5y한전 3y산금 1y+0.5 회사 AA-회사 BBB-크레딧 스프레드 변동 추이 (1M)회사채 등급별 스프레드1d5d1MAA-A+A0A- AA+AAAA- A++0.0 +0.0 +0.0 +0.0 +0.0 +0.0 +0.0 +0.0 여전채 등급별 스프레드1d5d1M+0.0 +0.0 +0.0 +0.0 +0.0 +0.0 +1.0 \'고금리 길어지네\'…회사채 발 끊고 은행 문 두드리는 기업들• 고금리 상태가 예상보다 길어지면서 기업들이 회사채 발행은 줄이고 은행 대출을 통해 자금을 조달하는 기조가 이어지고 있음• 연초 낮은 금리와 작년 하반기 레고랜드 사태로 지연된 발행 수요가 맞물려 발행시장이 강세였던 1분기와는 대조적 분위기• 회사채 순상환 기조가 지속된 기간동안 은행 대출은 늘어난 가운데, 이는 생각보다길어지고 있는 고금리 상황 때문으로 풀이연합뉴스(8/28)롯데케미칼, 등급강등 후폭풍…회사채 발행 \'냉랭\'• 롯데케미칼(AA/S)이 신용등급 강등에도 공모채 발행에 나섰지만, 시장의 냉랭한 분위기에 결국 발행액을 큰 폭 낮추기로 한 모습• 투자수요가 부족하다고 판단한 주관사들이 미매각을 피하기 위해서는 감액이 불가피하다고 요구한 까닭• 증권사 관계자는 "사실상 주관사들의 계열 물량 외에는 시장의 수요가 없어 불가피하게 물량을 감액한 것"이라 설명딜사이트(8/28)"현대차그룹, 재무안정성 우수…\'피크아웃\' 위험 낮아"• 한국신용평가가 현대자동차그룹이 우수한 재무안정성을 나타내고 있으며 일각에서 제기하는 \'피크아웃\' 우려가 현실화할 가능성은 크지 않다고 진단•"경쟁사의 생산 정상화에도 불구하고 판매 순위를 유지하고 있어 제고된 시장지위가 급격히 저하하지는 않을 것"이라는 평가• 상대적으로 부진한 실적을 내는 부품사의 경우 실적 변동성을 완화하고 비계열 수금융채 회사채 스프

In [36]:
# 데일리 리포트가 아닌 것만을 남김
df.drop(df[(df['증권사']=='유안타증권') & (df['텍스트'].str.startswith('(단위: 억원, %)'))].index, inplace=True)


In [ ]:
df[df['증권사']=='유안타증권'].iloc[0,2]

In [37]:
checklist.remove('유안타증권')
checklist

['교보증권',
 '하나증권',
 '유진투자증권',
 '키움증권',
 '한화투자증권',
 '이베스트증권',
 '신한투자증권',
 '메리츠증권',
 '하이투자증권',
 'DS투자증권',
 '대신증권',
 '미래에셋증권',
 '현대차증권',
 'IBK투자증권',
 'DB금융투자',
 '케이프투자증권',
 '이베스트투자증권',
 'LIG투자증권',
 'KTB투자증권',
 '이트레이드증권',
 '메리츠종금증권',
 '대우증권']

##### 대우증권

In [38]:
df[df['증권사']=='대우증권']

,증권사,날짜,텍스트
5275,대우증권,10.01.15,Daewoo Fixed Income Daily Daewoo 2010. 1. ...
5277,대우증권,10.01.14,Daewoo Fixed Income Daily Daewoo 2010. 1. ...
5279,대우증권,10.01.13,Daewoo Fixed Income Daily Daewoo 2010. 1. ...
5281,대우증권,10.01.12,Daewoo Fixed Income Daily Daewoo 2010. 1. ...
5283,대우증권,10.01.11,Daewoo Fixed Income Daily Daewoo 2010. 1. ...
...,...,...,...
5725,대우증권,08.11.27,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...
5726,대우증권,08.11.26,채권메모 - 2008. 11. 26. 수 서철수 02-768-3501 cybersc...
5727,대우증권,08.11.26,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...
5728,대우증권,08.11.25,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...


In [ ]:
df[df['증권사']=='유안타증권'].iloc[0,2]

In [41]:
df1=df

In [79]:
df[df['증권사']=='대우증권'].iloc[-1,2]

'채권메모 - 2008. 11. 24. 월 서철수 02-768-3501 cyberscs@bestez.com  절반의 지원과 절반의 영향력   한은이 ‘민간펀드’라는 원칙과 명분을 고수함에 따라, 채안펀드의 지원규모는 최대 50%로 제한될 수 밖에 없었다. 그러한 명분 아래에서는, 한은의 이번 지원은 나름대로는 최대한의 노력을 다한 것이라 평가할 수도 있겠다.    발권력을  통한  신규  유동성  주입의  첫사례라는  점이  이번  한은  지원의  의의이다.  아마도  앞으로  동  펀드의  증액과  그에  따른  한은  지원의확대 가능성을 배제할 수 없다는 판단이다. 다른 조건이 동일하다면, 동 펀드와 한은의 지원은 신용시장에 긍정적인 요인일 수 있을 것이다.   다만, 한은의 이번 지원에도 불구하고 구축효과에 대한 우려가 완전히 소멸되기는 힘들 수 있는 한편, 동 조치로 신용채권의 발행/유통이 얼마나 크게 개선될 수 있을지에 대한 불확실성이 일부 잔존하는 측면도 있다. 또한, 펀드조성이 단계적/순차적으로 진행될 것이고 보면 동 조치의 영향력이 무위험/신용 금리에 당장 큰 영향력을 미치기는 힘들 수 있다는 판단이다.  ■ 동 조치의 내용 -  채권펀드에  출자하는  금융기관에  최대  5조원,  출자금액의  50%까지  제공키로  결정.  산업은행/시중은행/보험  /증권사  등 참여 금융기관을 대상으로 국고채 직매입이나 통안증권을 중도 환매해주는 상대매매 방식.     가령, A금융기관이 2조원을 출자할 경우 한은이 A기관이 보유하고 있는 국고채를 직매입하거나 통안채 중도상환을 통해 1조원을 지원. 단,  펀드 규모가 10조원인 만큼, 한은의 총 지원규모는 최대 5조원이라는 것.    금융기관  요청시  RP지원  방식도  병행될  수  있으나,  주로  직매입/중도상환  방식으로  지원한다는  게  한은의  입장. 이는  한은의  발권력을  통해  금융시장에  직접적으로  신규  유동성이  주입되는  것으로,  단기  대출의  성격을  갖는 RP지원 방식 

In [53]:
# df1['텍스트']=df1.텍스트.str.split('이 자료에 게재된 내용들은 본인의 의견을 정확하게 반영하고 있으며 외부의 부당한 압력이나 간섭 없이 작성되었음을 확인함.').str[0]
# df1[df1['증권사']=='대우증권'].iloc[0,2]

'Daewoo Fixed Income Daily     Daewoo 2010. 1. 15. Fri 채권   윤일광(02-768-3058) ilkwang.yoon@dwsec.com 채권시황 (cid:122)  장 시작하자마자 외국인이 선물을 2000계약 이상 순매수하자 미 금리 상승으로 인한 부담이 줄어 금리는 보합 출발. 대부분 기관들이 향후 금리 향방에 대한 확신이 없는 상황에서 장중 내내 외국인의 강한 매수세에 끌려가는 양상이었음. 외국인의 매수세는 20일 이평선 상회 이후 기술적 움직임으로 설명되기도 하지만, 환율 하락에 힘입은 측면도 있는 듯함. 현물시장이 상대적으로 약세를 보여 금리 낙폭을 제한하였고, 단기물보다는 중장기물이 강세를 보였음. (cid:122)  ECB는 기준금리를 동결하였고, 현 수준이 적절하고 중기 물가 압력도 낮은 편이라고 해 당분간 금리조정에 나설 생각은 없는 듯. 미 소매판매는 예상치를 하회했는데, 높은 실업률로 인해 소비 회복이 만만치 않음을 보여줌. 미국채 30년물 입찰도 응찰률이나 해외 중앙은행 등 간접 입찰자들의 낙찰률이 높아 30년 금리는 9bp나 하락 (cid:122)  금일 역시 외국인의 선물 순매수가 이어질 것으로 예상되는 가운데, 국채 3년물 기준 4.2%를 하회할 지 관심. 다음 주 월요일 10년물 입찰에 대한 부담이 있을 수 있지만, 그 동안 10년물의 하락폭이 상대적으로 미미했다는 점에서 그다지 큰 영향은 없을 것으로 판단 관심 차트 및 경제지표 외국인 선물 대량 순매수로 금리 하락   소매판매 등 경제지표 부진으로 금리 하락 (%)통안채2년국고채5년국고채3년국고채10년기준금리미국2년미국10년미국3개월미국5년TIPS10년(%)자료: Bloomberg, 대우증권 리서치센터   자료: Bloomberg 중국 긴축 우려로 원자재 가격도 하락   RJ/CRB 지수 구성항목 비중 (1967=100)(1973=100)Reuter/Jefferies CRB 상품지수(L)달러 Index(R)축산물 7% - 생우 6% - 생

In [ ]:
df['텍스트']=df.텍스트.str.split('이 자료에 게재된 내용들은 본인의 의견을 정확하게 반영하고 있으며 외부의 부당한 압력이나 간섭 없이 작성되었음을 확인함.').str[0]
# df['텍스트']=df.텍스트.str.split('본 자료는').str[0]

In [80]:
checklist.remove('대우증권')
checklist

['교보증권',
 '하나증권',
 '유진투자증권',
 '키움증권',
 '한화투자증권',
 '이베스트증권',
 '신한투자증권',
 '메리츠증권',
 '하이투자증권',
 'DS투자증권',
 '대신증권',
 '미래에셋증권',
 '현대차증권',
 'IBK투자증권',
 'DB금융투자',
 '케이프투자증권',
 '이베스트투자증권',
 'LIG투자증권',
 'KTB투자증권',
 '이트레이드증권',
 '메리츠종금증권']

##### 메리츠종금증권

In [81]:
df[df['증권사']=='메리츠종금증권']

,증권사,날짜,텍스트
4072,메리츠종금증권,15.01.05,"Meritz China Macro Focus위기와 기회의 기로,2015년 중국경제의..."
4171,메리츠종금증권,14.07.14,Weekly ‘Macro & Bond’ Check 7월 셋째주 – 하반기 ...
4175,메리츠종금증권,14.07.07,Weekly ‘Macro & Bond’ Check 7월 둘째주 – 7월 금...
4180,메리츠종금증권,14.06.27,2014 3Q Meritz Asset Allocation세계경제 성장확대 재개 및한...
4192,메리츠종금증권,14.06.02,Weekly ‘Macro & Bond’ Check 6월 첫째주 – 6월 ...
...,...,...,...
5182,메리츠종금증권,10.09.16,Meritz Bond Daily _09/16 Fixed Income & Cr...
5185,메리츠종금증권,10.09.15,Meritz Bond Daily _09/15 Fixed Income & Cr...
5186,메리츠종금증권,10.09.14,Meritz Bond Daily _9/14 Fixed Income & Cre...
5190,메리츠종금증권,10.09.13,Meritz Bond Daily _09/13 Fixed Income & Cr...


In [82]:
df[df['증권사']=='메리츠종금증권'].iloc[0,2]

"Meritz China Macro Focus위기와 기회의 기로,2015년 중국경제의 향방은?Meritz ResearchEconomy & Bond Analyst오창섭6309-2713 / changsup.ough@meritz.co.krNWESwww.imeritz.com\x0c FTA 체결 및 위안화 직거래로 중국경제 회복은 한국에 가장 큰 수혜▶ 결론 : 한ㆍ중 FTA 체결 및 위안화 직거래 시장 개설 등으로 인해 중국경제 회복은 한국에 가장 큰 수혜 전망① 2015년은 중국의 제12차 5개년 계획의 마무리 해로 주택사업 및 지방정부 투자 활성화 등으로 경기개선 전망② 지난 중앙경제공작회의에서 시진핑이 경제안정화를 강조함에 따라 재정확장 및 통화완화 기조 강화 가능성③ 한ㆍ중 FTA 체결에 따른 소비재 및 통신 등 서비스업 호재와 중국 고정자산 투자증가에 따른 한국경제 수혜Focus I. 2015년은 12차 5개년 계획의 마무리 해▶ '부정부패' 척결 완료에 따른 경기부양 본격화- 지난해 신 4인방으로 불린 링지화 등 시진핑 라이벌들이 부정부패 수사로 몰락하며 시진핑 주석의 중앙집권화 완료- 이와 함께 지난 11월 2년 4개월만에 기준금리를 인하하는 등 하반기를 기점으로 중국정부의 경기부양 정책 본격화 양상- 2015년은 중국의 제12차 5개년 경제계획의 마무리 원년으로 주택사업 및 지방정부 투자 활성화 등으로 경기개선 전망- 지난 12월 중앙경제공작회의에서도 시진핑이 경제안정화를 강조함에 따라 올해 재정확장 및 통화완화 기조 강화 가능성Focus Ⅱ. 중국경제 회복은 한국에 가장 큰 수혜▶ 결론 : 지난해 11월 한/중 FTA 타결 및 12월부터 개장된 원-위안화 직거래 시장 개설로 중국경제 수혜 기대. 또한RQFII 획득으로 중국에 대한 금융투자 확대가 예상되는 가운데 한국의 위안화 역외 Hub 부각 가능성 주목▶ 중국은 한국의 수출/수입/무역흑자 1위국- 한국은 아시아 지역 유일의 위안화 직거래 시장인 가운데 한/중 FTA 체결로 중국경제 회복은 한국에 가

In [83]:
# df['텍스트']=df.텍스트.str.split('동 자료는 작성일 현재').str[0]

In [84]:
df[df['증권사']=='메리츠종금증권'].iloc[0,2]

"Meritz China Macro Focus위기와 기회의 기로,2015년 중국경제의 향방은?Meritz ResearchEconomy & Bond Analyst오창섭6309-2713 / changsup.ough@meritz.co.krNWESwww.imeritz.com\x0c FTA 체결 및 위안화 직거래로 중국경제 회복은 한국에 가장 큰 수혜▶ 결론 : 한ㆍ중 FTA 체결 및 위안화 직거래 시장 개설 등으로 인해 중국경제 회복은 한국에 가장 큰 수혜 전망① 2015년은 중국의 제12차 5개년 계획의 마무리 해로 주택사업 및 지방정부 투자 활성화 등으로 경기개선 전망② 지난 중앙경제공작회의에서 시진핑이 경제안정화를 강조함에 따라 재정확장 및 통화완화 기조 강화 가능성③ 한ㆍ중 FTA 체결에 따른 소비재 및 통신 등 서비스업 호재와 중국 고정자산 투자증가에 따른 한국경제 수혜Focus I. 2015년은 12차 5개년 계획의 마무리 해▶ '부정부패' 척결 완료에 따른 경기부양 본격화- 지난해 신 4인방으로 불린 링지화 등 시진핑 라이벌들이 부정부패 수사로 몰락하며 시진핑 주석의 중앙집권화 완료- 이와 함께 지난 11월 2년 4개월만에 기준금리를 인하하는 등 하반기를 기점으로 중국정부의 경기부양 정책 본격화 양상- 2015년은 중국의 제12차 5개년 경제계획의 마무리 원년으로 주택사업 및 지방정부 투자 활성화 등으로 경기개선 전망- 지난 12월 중앙경제공작회의에서도 시진핑이 경제안정화를 강조함에 따라 올해 재정확장 및 통화완화 기조 강화 가능성Focus Ⅱ. 중국경제 회복은 한국에 가장 큰 수혜▶ 결론 : 지난해 11월 한/중 FTA 타결 및 12월부터 개장된 원-위안화 직거래 시장 개설로 중국경제 수혜 기대. 또한RQFII 획득으로 중국에 대한 금융투자 확대가 예상되는 가운데 한국의 위안화 역외 Hub 부각 가능성 주목▶ 중국은 한국의 수출/수입/무역흑자 1위국- 한국은 아시아 지역 유일의 위안화 직거래 시장인 가운데 한/중 FTA 체결로 중국경제 회복은 한국에 가

In [ ]:
# 무엇을 더 할 수 있을까...

In [ ]:
df[df['증권사']=='메리츠종금증권'].iloc[1,2]

In [90]:
df[df['증권사']=='메리츠종금증권'].iloc[3,2]

"2014 3Q Meritz Asset Allocation세계경제 성장확대 재개 및한국 글로벌 자금유입Meritz ResearchEconomy & Bond Analyst오창섭02)6309-2713 / changsup.ough@meritz.co.krwww.imeritz.com\x0c메리츠종금증권 리서치센터\x0c 세계경제 성장확대 재개 및 한국 글로벌 자금유입 주목\uf0d8 결론 : 하반기 들어 세계경제 성장확대가 재개되는 가운데 자금흐름에서도 신흥국 및 위험자산 자금집중 전망\uf0d8 2014년 3Q Macro Overview- 내년까지 세계경제는 경기사이클에 있어 확장국면이 진행되는 가운데 하반기 본격적인 성장확대 재개 전망- 선진국 경제는 세계경제 성장을 주도하고 있는 가운데 미국의 부동산경기 및 유럽의 소비경기 회복 가능성 주목- 신흥국 경제는 1/4분기를 경기저점으로 중국이 회복세를 나타내는 가운데 원자재 수요증가에 따른 수혜 기대\uf0d8 2014년 3Q Currency Market- 글로벌 외환시장에서는 미국과 중국 등을 중심으로 통화정책 차별화가 나타나는 가운데 환율전쟁 가시화- 미국은 연준의 양적완화 축소가 지속되고 있는 가운데 내년 상반기를 기점으로 기준금리 인상 가능성 부각- 반면 상반기중 중국과 유럽이 금융완화 기조로 전환함에 따라 환율에 있어 미국과 중국 등의 이해상충 심화\uf0d8 2014년 Global Money Flow- 지역별 자금유입에 있어 지난해 미국으로 집중됐던 글로벌 자금이 올해 들어 아시아 및 유럽으로 자금유입 강화- 상품별로는 올해 들어 대기자금 성격의 MMF에서 자금이 크게 유출된 가운데 주식 및 채권으로 동반 자금유입 진행- 국내에서는 최근 주식시장에서 자금유출 완화 조짐이 나타나는 가운데 외국인의 국내주식 및 국내채권 순투자 전환\uf0d8 2014년 3Q Asset Allocation Strategy- 글로벌펀드 시가총액 기준 표준 자산비중은 주식&채권&부동산&원자재&대안투자 수준- 국내펀드 설정잔액 기준 

In [85]:
checklist.remove('메리츠종금증권')
checklist

['교보증권',
 '하나증권',
 '유진투자증권',
 '키움증권',
 '한화투자증권',
 '이베스트증권',
 '신한투자증권',
 '메리츠증권',
 '하이투자증권',
 'DS투자증권',
 '대신증권',
 '미래에셋증권',
 '현대차증권',
 'IBK투자증권',
 'DB금융투자',
 '케이프투자증권',
 '이베스트투자증권',
 'LIG투자증권',
 'KTB투자증권',
 '이트레이드증권']

##### 이트레이드증권

In [86]:
df[df['증권사']=='이트레이드증권']

,증권사,날짜,텍스트
4017,이트레이드증권,15.03.09,인도로 가야 안전합니다 / The Weekly 집 팔아서 인도 채권 사...
4019,이트레이드증권,15.03.02,"Part VI 춘삼월, 싱숭생숭 알수없는 크레딧 시장 섹터 / 산업분석 ..."
4022,이트레이드증권,15.02.23,"통화정책회의, , , , 첫 의사록 공개 . ECB 통화정책..."
4027,이트레이드증권,15.02.13,2015-02-13FI 권규백Credit 손소현spri@etrade.co.krthe...
4028,이트레이드증권,15.02.12,2015-02-12FI 권규백Credit 손소현spri@etrade.co.krthe...
4029,이트레이드증권,15.02.10,2015-02-10FI 권규백Credit 손소현spri@etrade.co.krthe...
4035,이트레이드증권,15.02.09,2015-02-09FI 권규백Credit 손소현spri@etrade.co.krthe...
4036,이트레이드증권,15.02.06,2015-02-06FI 권규백Credit 손소현spri@etrade.co.krthe...
4037,이트레이드증권,15.02.05,2015-02-05FI 권규백Credit 손소현spri@etrade.co.krthe...
4039,이트레이드증권,15.02.03,Fixed Income & FX권규백spri@etrade.co.kr이트레이드증권 ...


In [91]:
df[df['증권사']=='이트레이드증권'].iloc[0,2]

'인도로  가야  안전합니다  / The Weekly 집  팔아서  인도  채권  사라 크레딧    Credit  손소현  02.3779-0055 thecredit@ebestsec.co.kr 용을 이긴  코끼리 지난  2월  9일  인도  정부는  2014년  4분기  GDP성장률이  전년동기비  7.5%를  기록했다고  밝혔다.  2014회계연도  GDP성장률  전망치는  6.9%에서  7.4%로  상향  조정했다.  성장률  상향  조정은  GDP집계시  기준  연도를  2004회계연도에서  2013회계연도로  바꾼 것의  영향이  컸다.  하지만  기준  연도  변경을  차치하고서라도  인도  경제의  성장세는  명백한 사실이다.  얼마  전  전인대에서  중국은  올해  GDP성장률  목표치를  7.0%로  하향  조정했지만,  주요 기관들의  인도  GDP성장률은  속속  상향  조정되고  있다.  인도는  바야흐로  인구  3억  이상  초대형국  가운데  가장  성장률이  높은  나라로  자리매김했다.  말  그대로  용을  이긴  코끼리가 등장했다.  그림1  기존의  GDP  산정방식으로도  인도는  중국을  추월 중국인도\'13\'14\'15\'16자료: 무디스, 이 베스트투자증권 리서치본부 지금 당장  인도  채권을  사라 그렇다면  그  어떤  때보다  빠르게  달리고  있는  인도  코끼리에  올라탈  방법은  무엇인가. 인도 채권을 사는 것이다.  연내  기준금리  추가  인하  가능성  상존 올해  초  당사의  투자전략팀에서  ‘인사동’이라는  자료를  통해  올해  중국과  인도,  인도네시아를  주목하라고  할  때만  해도  인도의  기준금리는  8%였다.  인도중앙은행  총재  라구잠  라잔이  2013년에  취임한  이후  두자릿수대를  넘나드는  인도의  소비자물가상승률을 잡기 위해 기준금리를 세 차례나 인상한 결과였다.  이  베스트투자증권  리서치본부  1       \x0cThe Weekly /  인도로  가야  안

In [93]:
checklist.remove('이트레이드증권')
checklist

['교보증권',
 '하나증권',
 '유진투자증권',
 '키움증권',
 '한화투자증권',
 '이베스트증권',
 '신한투자증권',
 '메리츠증권',
 '하이투자증권',
 'DS투자증권',
 '대신증권',
 '미래에셋증권',
 '현대차증권',
 'IBK투자증권',
 'DB금융투자',
 '케이프투자증권',
 '이베스트투자증권',
 'LIG투자증권',
 'KTB투자증권']

##### KTB투자증권

In [95]:
df[df['증권사']=='KTB투자증권']

,증권사,날짜,텍스트
4006,KTB투자증권,15.04.30,Credit Analyst 김민정☎ 2184-2377 mj.kim@ktb.co.kr...
4065,KTB투자증권,15.01.07,Morning Insight 미 국채수익률 어디로? 안전자산 함정….10년물 ...
4099,KTB투자증권,14.11.21,21 Nov. 20142015 Credit Outlook계영배의 품격Credit ...
4194,KTB투자증권,14.05.28,28 May. 20142014년 하반기 크레딧시장 전망대담한 유동성과 선택적 기회...
4205,KTB투자증권,14.04.29,MonthlyCredit Gravity29 Apr. 2014최고보다는 만족Cred...
...,...,...,...
5150,KTB투자증권,10.10.11,2010년 10월 11일 복권과 보험 ▶ 지난 주 시장 요약 • 금리 인상 경계...
5161,KTB투자증권,10.10.04,러너스 하이 2010년 10월 4일 ▶ 지난 주 시장 요약 • 8 / 9월 ...
5176,KTB투자증권,10.09.27,2010년 9월 27일 윈도우 드레싱과 쇼핑▶ 지난 주 시장 요약 • 9월 금통...
5188,KTB투자증권,10.09.13,2010년 9월 13일 무지개의 끝자락 ▶ 지난 주 시장 요약 • 9월 금통위...


In [96]:
df[df['증권사']=='KTB투자증권'].iloc[0,2]

'Credit Analyst 김민정☎ 2184-2377 mj.kim@ktb.co.kr연구원 이남석☎ 2184-2296 nslee@ktb.co.kr30 Apr.  2015MonthlyCredit Gravity장기 구간 중심 리스크관리 강화 필요채권금리 상승과 수급 부담 등으로 장기 구간 중심 약세 가능성에 주의● 국내 경기 회복 기대감과 주식시장 강세 등에 따라 채권금리 상승세● 6월말까지 안심전환대출 MBS 34조 원 발행예정, 장기 구간 중심 수급 부담● 회사채와 여전채 신용등급 하향조정에도 불구, A급 회사채 투자매력도 유효특수채 – 안심전환대출 MBS 발행 증가에 따른 구축효과 유의● 안심전환대출 유동화 MBS 본격적으로 발행 시작, 가이드라인 윤곽● 7년 만기 이하 안심전환대출자산 양호한 은행이 지분대로 의무매입● 10년 만기 이상 10~14조 원 시장입찰방식을 통해 발행, 단기적 수급 부담  은행채 – 안심전환대출 시행은 은행 수익성 부담 요인● 안심전환대출 시행, 고마진자산 감소 및 MBS 매입으로 수익성 저하 불가피● 7년 이하 비경쟁입찰로 20~24조 원 안심전환대출 MBS 의무 매입● 리스크관리나 자본적정성, 자산성장성 측면 긍정적 효과 기대여전채 – 금융지주/제조업계열 여전사 신용도 차별화 지속● 발행시장 강세에도 불구, 차익실현에 따른 약세 지속● 신평사 정기평정 결과, 효성/두산캐피탈 신용등급 하락으로 일단락● 재무적 지원능력과 계열 위험 감안시, 금융지주/제조업계열 여전사 차별화회사채 – 신용등급 하락에도 불구, 우호적 발행환경 지속● 포스코 계열사와 조선, 정유, 건설업을 중심으로 신용등급 하향조정 증가● 정유, IT 등 일부 실적 개선, 등급 하락에도 불구 자금조달 순조롭게 진행● AA급 가격 부담, 고금리 투자매력도가 유효한 A급 회사채 추가 강세 지속\x0cCONTENTSI. Summary & Key chartsII. 크레딧 투자전략특수채 – 안심전환대출 MBS 발행 증가에 따른 구축효과 유의은행채 – 안심전환대출 시행은 은행 수

In [97]:
df['텍스트']=df.텍스트.str.split('이 페이지는 편집상 공백입니다.').str[0]


In [98]:
checklist.remove('KTB투자증권')
checklist

['교보증권',
 '하나증권',
 '유진투자증권',
 '키움증권',
 '한화투자증권',
 '이베스트증권',
 '신한투자증권',
 '메리츠증권',
 '하이투자증권',
 'DS투자증권',
 '대신증권',
 '미래에셋증권',
 '현대차증권',
 'IBK투자증권',
 'DB금융투자',
 '케이프투자증권',
 '이베스트투자증권',
 'LIG투자증권']

##### LIG투자증권

In [99]:
df[df['증권사']=='LIG투자증권']

,증권사,날짜,텍스트
3203,LIG투자증권,16.12.30,2016/12/29 트럼프가 행할 무역정책은 채권시장에 우호적 CAPE Fi...
3212,LIG투자증권,16.12.16,LIG Research Division ▶▶▶ Fixed Income Analys...
3229,LIG투자증권,16.11.28,LIG Research Center 2016/11/25 LIG Fixed Incom...
3241,LIG투자증권,16.11.14,LIG Research Center 2016/11/11 LIG Fixed Incom...
3268,LIG투자증권,16.10.14,LIG Research Division ▶▶▶ Fixed Income Analys...
3299,LIG투자증권,16.09.12,LIG Research Division ▶▶▶ Fixed Income Analys...
3307,LIG투자증권,16.09.05,LIG Research Center 2016/09/02 LIG Fixed Incom...
3330,LIG투자증권,16.08.16,LIG Fixed Income Weekly LIG Research Center 20...
3333,LIG투자증권,16.08.12,LIG Research Division ▶▶▶ Fixed Income Analys...
3340,LIG투자증권,16.08.08,LIG Fixed Income Weekly LIG Research Center 20...


In [102]:
df[df['증권사']=='LIG투자증권'].iloc[0,2]

"2016/12/29 트럼프가  행할  무역정책은  채권시장에  우호적 CAPE Fixed Income Weekly Fixed Income Analyst 최운선   02)6923-7344   Run4u@capefn.com 국내  채권시장  리뷰 \uf09f  환율 상승에도 외국인 수급 개선은 금리하락에 기여함 \uf09f  환율  상승:  1,200원을  돌파하는  환율은  투자심리에  부담이  되고  있음.  환율  상승의  외부요인으로는  미국의  금리인상과  중국의  외환보유고  감소가  이머징  투자에  대한 회피  심리를  지지하고  있기  때문임.    내부요인으로는  연초  예정된  헌재의  탄핵관련  결정에  대한  정치적인  불확실성을 겨냥한  투기적인  원화  약세  투자가  형성되어  있기  때문임 \uf09f  부동산  관련  투자의  위축  기대감은  안도감을  제공:  모기지금리의  상승은  부동산에  대한  투자수요의  위축  가능성을  보여주고  있음.  최근,  미국의  부동산  지표는  두  가지  특징을  반영하고 있음.  먼저,  모기지  금리가  추가적으로  상승하기  전에  선취매  하고자  하는  거래가  신규주택에 대한  거래에서  발생하고  있음.  반면,  30년  모기지  대출  금리가  4.0%대에  진입함에  따라  기존주택을  거래하는  신규  모기지  신청은  둔화되기  시작함.  부동산  투자수요  둔화를  주목하는  것은  인플레이션에  대한  기존의  우려를  다소  완화시킬  것으로  기대하기  때문임 \uf09f  BOJ  디플레이션  탈피  선언:  구로다  총재의  디플레이션  탈피  선언은  BOJ의  금융완화  정책이 추가적으로  확대될  가능성이  낮아졌음을  시사함.  이는  외환시장에서  엔캐리  확대보다  엔캐리의  청산  모멘텀이  될  것으로  보임.  즉,  엔/달러  상승에  대한  기대감을  약화시킬  것임 주간  채권시장  전망 \uf09f  전망: 환율 상승이 제한되면 

In [ ]:
'본  조사자료'

In [103]:
df[df['증권사']=='LIG투자증권'].iloc[1,2]

'LIG Research Division  ▶▶▶ Fixed Income Analysis  ■  2016/12/15 Fixed Income/Credit Analyst 최운선ㆍ02)6923-7344ㆍrun4u@ligstock.com LIG Monetary Policy View 기준금리  동결  지속될  전망 2017년  1분기에도  기준금리는  동결될  것으로  전망된다. 15일  한국은행은  기준금리를  1.25%로  6개월  연속  동결을  결정했다.  동결배경으로  거론된  FRB의  통화정책  정상화,  미국  신정부  출범과  정책에  대한 불확실성,  그리고  이러한  대외  요인에  의한  외국인  자금의  이탈  가능성은  관련된  우려가  완화되기  전에  경기친화적인  정책으로  선회할  가능성이  낮음을  시사한다.  또한,  국내의  경우  유가상승과  이로  인한  수입물가  상승이  물가상승을  견인할  것이며,  가계부채  증가의  둔화에  대한  확인이  요구되기  때문이다.  6개월  연속  1.25%로  동결 \uf09f  금리동결  배경:  국내  경기의  성장경로의  하방위험이  증대된  것으로  평가하고  있으나,  금융안정  리스크에  더  주목했음.  금융안정  리스크  중  대외요인은  먼저  FRB의  통화정책  정상화가  어느정도  속도와  폭으로  진행될  것인지에  대한  불확실성과  미국  트럼프  행정부  출범  후  실행될  정책에  대한  우려를  들  수  있음.  국내  리스크  요인으로는  가계부채  증가와  수입물가  상승으로  인한  물가상승임   내수의  개선  움직임이  미약하다! \uf09f  세계경제:  미국이  견조한  회복세를  지속하고  있고,  중국은  완만한  성장세를  유지하며  유로지역도  개선움직임이  있는  것으로  판단함.  즉,  해외경제의  성장에  대한  신뢰를  보여줌 \uf09f  국내경제:  해외  경제  성장이  연말  글로벌  소비시즌과  중첩되면서  수출이

In [104]:
checklist.remove('LIG투자증권')
checklist

['교보증권',
 '하나증권',
 '유진투자증권',
 '키움증권',
 '한화투자증권',
 '이베스트증권',
 '신한투자증권',
 '메리츠증권',
 '하이투자증권',
 'DS투자증권',
 '대신증권',
 '미래에셋증권',
 '현대차증권',
 'IBK투자증권',
 'DB금융투자',
 '케이프투자증권',
 '이베스트투자증권']

##### 이베스트투자증권

In [105]:
df[df['증권사']=='이베스트투자증권']

,증권사,날짜,텍스트
3015,이베스트투자증권,17.07.03,Fixed Income 전병하 byunghajeon@ebestsec.co.kr V...
3022,이베스트투자증권,17.06.26,Fixed Income 전병하 byunghajeon@ebestsec.co.kr V...
3028,이베스트투자증권,17.06.20,Vol. 2 2017. 6. 20 채권퉁 시행의 가시화 중국 크레딧채권 ...
3029,이베스트투자증권,17.06.20,Fixed Income 전병하 byunghajeon@ebestsec.co.kr V...
3035,이베스트투자증권,17.06.15,"채권전병하byunghajeon@ebestsec.co.kr연준, 금리인상과 더불어 자..."
...,...,...,...
3987,이베스트투자증권,15.06.25,2015-06-25Credit 손소현thecredit@etrade.co.krComm...
3988,이베스트투자증권,15.06.24,2015-06-24Credit 손소현thecredit@etrade.co.krComm...
3989,이베스트투자증권,15.06.23,2015-06-23Credit 손소현thecredit@etrade.co.krComm...
3992,이베스트투자증권,15.06.22,2015-06-22Credit 손소현thecredit@etrade.co.krComm...


In [106]:
df[df['증권사']=='이베스트투자증권'].iloc[0,2]

"Fixed Income  전병하 byunghajeon@ebestsec.co.kr Vol 19.   2017년  7월  03일 Fixed Income Weekly   7월  1주차  시장전망 Part I  주간  Review & Preview   Part II  Weekly Check Point 이베스트투자증권  리서치센터  1      \x0cFixed Income  전병하 byunghajeon@ebestsec.co.kr Fixed Income Weekly   Executive Summary Review:    국내:  주간  국고채  3년물은  1.688%에서  1.691%로  상승했으며  국고채  10년물은  2.129%에서  2.208%로  상승하였다.  장단기금리차는  지난주의  44.1bp에서  51.7bp로  확대되면서  금리커브가  최근의  플래트닝에서  스티프닝으로  전환되었다.  주중  외국인  매도세가  급증하면서  국내  장단기금리  모두  강한  상승세를  보였다.  외국인이  순매도로  전환된  구간은  5년  이하의  상대적  단기  채권이었고  국채 현물을 1.8조원  정도를  순매도  했고  통안채를 0.2조원  정도  순매도  했다.     선진국:  미국  시장에서는  경제지표가  예상보다  부진한  모습이  보이면서  주  초반에는  최근  가장  낮은  금리인  2.135%에  거래되었다.  하지만  드라기  총재의  발언이  금리상승의  주요한  트리거로  작용했다.  주  후반  미국  GDP  3rd  발표에서  성장률  확보치가  연율1.2%에서  1.4%로  0.2%p  상향되면서 채권금리를  보다  상승시켰다.  유로존과  일본의  채권금리는  드라기총재의  발언  이후로  강한  상승세를  보이고  있다.     신흥국:  지난  한  주간  신흥국  채권시장은  드라기  총재의  발언에  민감하게  반응하는  모습이었다.  신흥국을  제외하고는  캐나다와  영국  등  금리인상을  염두에  둔  국가들의  금

In [107]:
df[df['증권사']=='이베스트투자증권'].iloc[0,2]

"Fixed Income  전병하 byunghajeon@ebestsec.co.kr Vol 19.   2017년  7월  03일 Fixed Income Weekly   7월  1주차  시장전망 Part I  주간  Review & Preview   Part II  Weekly Check Point 이베스트투자증권  리서치센터  1      \x0cFixed Income  전병하 byunghajeon@ebestsec.co.kr Fixed Income Weekly   Executive Summary Review:    국내:  주간  국고채  3년물은  1.688%에서  1.691%로  상승했으며  국고채  10년물은  2.129%에서  2.208%로  상승하였다.  장단기금리차는  지난주의  44.1bp에서  51.7bp로  확대되면서  금리커브가  최근의  플래트닝에서  스티프닝으로  전환되었다.  주중  외국인  매도세가  급증하면서  국내  장단기금리  모두  강한  상승세를  보였다.  외국인이  순매도로  전환된  구간은  5년  이하의  상대적  단기  채권이었고  국채 현물을 1.8조원  정도를  순매도  했고  통안채를 0.2조원  정도  순매도  했다.     선진국:  미국  시장에서는  경제지표가  예상보다  부진한  모습이  보이면서  주  초반에는  최근  가장  낮은  금리인  2.135%에  거래되었다.  하지만  드라기  총재의  발언이  금리상승의  주요한  트리거로  작용했다.  주  후반  미국  GDP  3rd  발표에서  성장률  확보치가  연율1.2%에서  1.4%로  0.2%p  상향되면서 채권금리를  보다  상승시켰다.  유로존과  일본의  채권금리는  드라기총재의  발언  이후로  강한  상승세를  보이고  있다.     신흥국:  지난  한  주간  신흥국  채권시장은  드라기  총재의  발언에  민감하게  반응하는  모습이었다.  신흥국을  제외하고는  캐나다와  영국  등  금리인상을  염두에  둔  국가들의  금

In [108]:
df[df['증권사']=='이베스트투자증권'].iloc[-2,2]

'2015-06-22Credit 손소현thecredit@etrade.co.krComment 그리스-채권단, 유로존 정상회의 앞두고 접점 모색알렉시스 치프라스 그리스 총리는 상호 호혜적인 새 협상안을제시했으며, 22일 유로존 재무장관 회의가 분수령이 될 것으로예상. 한편 UBS는 그렉시트가 발생하면 동유럽 국가들이 가장큰 타격을 입을 것이며, 동유럽 통화는 15~20% 급락할 것이라고전망 우크라이나, 헤어컷 조건으로 성장연계 채권발행 제시우크라이나는 민간 채권단에 40% 헤어컷을 요구했지만, 채권단은 상환 연장과 이자 감축 제안. 이에 우크라이나는헤어컷에 동의하는 조건으로 성장연계 채권을 발행하는 방안을제시. 2001년 아르헨티나 채무 위기 해결에 사용했던 방식 저금리 후폭풍, 가계 기업 은행 빚 1,300조원 돌파21일 한국은행에 따르면 예금은행의 대출금은지난 15일 현재 1,302조원으로 집계. 이 중 가계 빚이 600조원, 기업 빚이 700조원 규모. 금리인하와 부동산 규제 완화로주택담보대출을 비롯한 가계대출이 큰 폭으로 증가했으며, 기술금융 확대로 중소기업 대출이 증가한 영향. 금리가오르거나 사업이 부실해져 대출 상환에 문제가 생기면금융권의 부실로 이어질 우려가 커지고 있어 정부는가계부채관리 강화방안을 마련하고 내달 중 구체적인 방안을발표하기로 한 상황1. 국내 채권시장 동향 및 자금흐름국내금리스프레드3년/5년3년/10년5년/10년FX/IRS/CRSFX/IRS/CRSFX/IRS/CRSFX/IRS/CRSCD통안 1년통안 2년국고 3년국고 5년국고 10년국고 30년 1D1D1D6/176/176/176/17N/A주: 자금흐름, 투자자별 순매수종가종가종가6/186/186/186/18N/A원/달러원/100엔IRS 1년IRS 3년IRS 10년CRS 1년CRS 3년CRS 10년자금흐름MMF혼합형채권형채권관련자금3. 채권시장 주요 일정월6/221M1M1M6/156/156/156/151W1W1W6/166/166/166/16화6/232. 국내 크레딧시장 동향 및 투자자별 

In [109]:
checklist.remove('이베스트투자증권')
checklist

['교보증권',
 '하나증권',
 '유진투자증권',
 '키움증권',
 '한화투자증권',
 '이베스트증권',
 '신한투자증권',
 '메리츠증권',
 '하이투자증권',
 'DS투자증권',
 '대신증권',
 '미래에셋증권',
 '현대차증권',
 'IBK투자증권',
 'DB금융투자',
 '케이프투자증권']

##### 이베스트증권

In [110]:
df[df['증권사']=='이베스트증권']

,증권사,날짜,텍스트
25,이베스트증권,23.08.25,2016년 6월 23일 Investment Strategy 투자코멘트 | 2...
46,이베스트증권,23.08.23,Today Chart BondFixed Income 우혜영 _ 02 3779 863...
47,이베스트증권,23.08.23,Today Chart BondFixed Income 우혜영 _ 02 3779 863...
200,이베스트증권,23.07.27,2016년 6월 23일 sInvestment Strategy 투자코멘트 | ...
211,이베스트증권,23.07.26,Today Chart BondFixed Income 우혜영 _ 02 3779 863...
...,...,...,...
2993,이베스트증권,17.07.21,7월 ECB는 9월 양적완화 축소에 대한 가능성을 열어두었다 채권 전병하 byun...
2995,이베스트증권,17.07.17,Vol 21. 2017년 7월 17일 Fixed Income 전병하 연준의...
3001,이베스트증권,17.07.14,금통위 Review: 성장률은 올라가지만 금리는 아직 한국은행은 7월 기준금리...
3003,이베스트증권,17.07.13,옐런의 발언이 진짜 비둘기파적이었을까? 채권 전병하 byunghajeon@ebes...


In [111]:
df[df['증권사']=='이베스트증권'].iloc[0,2]

'2016년  6월  23일 Investment Strategy  투자코멘트  | 2023. 8. 24 8월  금통위  Review: 잊지  말자, Mandate 물가  >  금융안정  >  경기 Fixed Income  우혜영 02 3779 8630 _whydele@ebestsec.co.kr 당분간  통화정책의  방점은  금융안정에 8월  금융통화위원회에서는  만장일치로  기준금리를  3.50%로  동결했다.  통화정책방향문구에서  주목할만한  부분은  ①現  긴축기조를  유지한  배경으로  ‘주요국  경기  관련한  불확실성이  높아졌다’는  문구  추가,  ②5월  통방문과  비교하면  향후  물가경로  리스크에  ‘기상여건’  문구  추가  및  ‘공공요금  추가  인상  여부’  문구  삭제,  ③정책방향  문구에서  인플레이션  둔화  ‘속도’ →  ‘흐름’으로  문구  수정  및  ‘가계부채  증가  추이’  문구  삽입  등이었다.  최종금리에  대해서는  역시나  모든  금통위원이  3.75%로  가져갈  가능성을  열어두고  있었다.   지난  회의에서  언급된  인상  가능성을  열어둔  이유는  ①연준  통화정책  불확실성에  따른  외환시장  변동성  확대  가능성,  ②목표를  상회하는  高  근원물가  상승률의  전개  확인  필요,  ③가계부채  추이  불확실성이었다.  이번에는  ①과  ③이  비슷하게  언급됐지만  물가에 대해서는  외환시장  변동성  확대로  물가  변동성이  확대될  수  있음을  지적한  정도였다.  이날  기자회견에서  이창용  총재는  한국은행에게는  물가가  가장  중요하고  두  번째는  금융안정이라면서  Mandate를  강조했다.  지난  22일,  국회  기획재정위원회  현안질의에서는  다른  나라보다  물가  안정  성과가  좋다고  평가한  반면,  가계부채  증가세가  계속되면  문제가  될  수  있다면서  더  이상 늘어나지  않도록  미시적·거시적  조치에  나서겠다는  입장을  보이

In [112]:
df[df['증권사']=='이베스트증권'].iloc[1,2]

'Today Chart BondFixed Income 우혜영 _ 02 3779 8630  whydele@ebestsec.co.krChart of the Day: 8월 금통위 Preview, 매파적인 동결의 연속수정경제전망: 근원물가 상승률 상향조정 예상되나 영향은 제한적① 실제 및 기대물가상방 리스크최근 국제유가 상승 + 에너지 기저효과 소멸 등으로 헤드라인 물가상승률반등 불가피. 개인서비스물가 상승률이 둔화되곤 있으나 여전히 4.6%대의 높은 상승률 지속. 일반인 기대인플레이션도 3.3%로 3% 상회 중② 환율 변동성 확대 등외환부문 불안 재현7월 FOMC 의사록 공개 후 美 연준의 연내 추가 인상 가능성 다시금 언급되고 있음. 연준 정책 관련 기대 변화에 달러인덱스 강세가 심화된다면최근의 원화 자체 약세에 더해져서 원달러 환율 단기간에 급등할 가능성③ 가계대출 재증가 및금융불균형 해소 지연부동산 대출 관련 규제 완화, 일부 대출금리 하락에 기인해 주담대 중심의가계부채 재상승. GDP대비 가계부채 비중이 작년 수주으로 재차높아지는 등 상승 추세 계속된다면 추가 인상으로 대응할 필요성 존재CPI 석유류수입물가 원유현재中 부동산 이슈7월 FOMC기타대출주택담보대출예금취급기관 가계대출원/엔원/파운드원/유로원/달러달러인덱스파운드/달러22/0722/1123/0319/0120/1022/07유로/달러달러/엔GDP 경제성장률물가상승률근원물가Key Point● 기준금리를 만장일치로 3.50%에서 동결하면서도 모든 금통위원들이3.75%로 인상할 가능성을 열어둬야 한다는 입장 재확인할 것. 7월 회의때와 같은 이유를 근거로 추가 인상 필요성 언급 전망●최근 여타 국가들의 통화 대비 원화의 평가절하가 두드러짐에 따라 원화약세 흐름 확인. 7월 FOMC 의사록 공개 후 9월 동결 4분기 중 추가 인상가능성 소폭 높아짐. 이는 외환부문 불안 재현될 가능성 배제 불가● 주담대를 중심으로 가계부채가 지난 5월부터 2개월 연속 증가한 점 역시긴축기조 지속 필요성을 높이고 있다고 판단주: 

In [113]:
checklist.remove('이베스트증권')
checklist

['교보증권',
 '하나증권',
 '유진투자증권',
 '키움증권',
 '한화투자증권',
 '신한투자증권',
 '메리츠증권',
 '하이투자증권',
 'DS투자증권',
 '대신증권',
 '미래에셋증권',
 '현대차증권',
 'IBK투자증권',
 'DB금융투자',
 '케이프투자증권']

##### 유진투자증권

In [114]:
df[df['증권사']=='유진투자증권']

,증권사,날짜,텍스트
2,유진투자증권,23.08.30,Eugene’s FICC Update Fixed Income 김지나_ 02)368-...
10,유진투자증권,23.08.29,Eugene’s FICC Update Fixed Income 김지나_ 02)368-...
19,유진투자증권,23.08.28,Eugene’s FICC Update Fixed Income 김지나_ 02)368-...
29,유진투자증권,23.08.25,Eugene’s FICC Update Fixed Income 김지나_ 02)368-...
39,유진투자증권,23.08.24,Eugene’s FICC Update Fixed Income 김지나_ 02)368-...
...,...,...,...
1422,유진투자증권,22.12.20,Eugene’s FICC Update Fixed Income 김지나_ 02)368...
1426,유진투자증권,22.12.19,Eugene’s FICC Update Fixed Income 김지나_ 02)368...
1432,유진투자증권,22.12.16,Eugene’s FICC Update Fixed Income 김지나_ 02)368...
1435,유진투자증권,22.12.15,Eugene’s FICC Update Fixed Income 김지나_ 02)368...


In [115]:
df[df['증권사']=='유진투자증권'].iloc[0,2]

'Eugene’s FICC Update Fixed Income 김지나_ 02)368-6149_ jnkim0526@eugenefn.com  Economist 이정훈_ 02)368-6176_ jhoonlee@eugenefn.com Fixed Income  외국인 3년 국채선물 수급 외국인 10년 국채선물 수급 8/29일 1D 한국 국고채 3년물 1W YTD 국내채권 해외채권 한국 국고채 10년물 장단기금리차 3년 국채 선물 10년 국채선물 미국채 2년물 미국채 10년물 장단기금리차 독일국채 10년물 호주국채 10년물 일간 금리 등락 폭 & 커브 국내 채권시장 동향 전망 및 향후 주요 이벤트 \uf06e  국내  채권시장  불  플래트닝  마감.  투자심리  훼손  점차 \uf06e  미국  채권시장  강세  마감.  고용시장  둔화에  대한  기대회복 중. 감. \uf06e  기획재정부가  발표한  2024년  예산안에  따르면  국고채 \uf06e  PCE  물가와  비농업  고용지표를  앞두고  발표된  구인구발행이  158.8조원으로  올해에  비해  9조원  줄어든다는 직보고서에서  고용시장  둔화의  조짐을  발견.  채용공고것을 확인하면서 수급적 호재로 인식.  가 28개월 만에 최저를 기록하면서 긴축 부담감 축소. \uf06e  미국  고용과  물가를  앞둔  경계심이  상존했지만  절대금\uf06e  금일  장중  호주와  독일  물가  발표가  예정.  미국  8월 리  메리트와  최근  금리  상승세에  따른  피로감으로  저가ADP  민간고용  결과에도  미국  금리  변동성  확대  가능성 매수 유입되면서 금리 하락 지속. 있음. 자료 출처: 연합인포맥스, Refinitiv, 유진투자증권 '

In [116]:
# 깔끔 기본내용

In [117]:
checklist.remove('유진투자증권')
checklist

['교보증권',
 '하나증권',
 '키움증권',
 '한화투자증권',
 '신한투자증권',
 '메리츠증권',
 '하이투자증권',
 'DS투자증권',
 '대신증권',
 '미래에셋증권',
 '현대차증권',
 'IBK투자증권',
 'DB금융투자',
 '케이프투자증권']

##### 케이프투자증권

In [118]:
df[df['증권사']=='케이프투자증권']

,증권사,날짜,텍스트
3000,케이프투자증권,17.07.14,CAPE Research Division ▶▶▶ Fixed Income Analy...
3045,케이프투자증권,17.06.09,| 2017 년 06월 | 2017년 채권시장 전망과 투자 아이디어 | 2017년 ...
3064,케이프투자증권,17.05.26,CAPE Research Division ▶▶▶ Fixed Income Analy...
3072,케이프투자증권,17.05.19,CAPE Research Division ▶▶▶ Analysis _ 2017/05...
3113,케이프투자증권,17.04.14,CAPE Research Division ▶▶▶ Fixed Income Analy...
3125,케이프투자증권,17.03.27,2017/03/24 월말 지표호전 부담되나 필립스 커브 평탄화를 주목 ...
3130,케이프투자증권,17.03.20,2017/03/17 박스권 재진입 후 세제 개혁안을 기다리는 관망 CAP...
3149,케이프투자증권,17.02.24,CAPE Research Division ▶▶▶ Fixed Income Analy...
3159,케이프투자증권,17.02.13,2017/02/10 트럼프 정책에 대한 저항이 투자심리에 긍정적 CAPE ...
3187,케이프투자증권,17.01.16,CAPE Research Division ▶▶▶ Fixed Income Analy...


In [119]:
df[df['증권사']=='케이프투자증권'].iloc[0,2]

"CAPE Research Division  ▶▶▶ Fixed Income Analysis _ 2017/07/13 Fixed Income/Credit Analyst  최운선ㆍ02)6923-7344ㆍrun4u@capefn.com CAPE Monetary Policy View 8월에도  동결국면  유지될  전망 한국은행은  국내경제가  견실한  성장세를  나타낼  것으로  예상하고  있으나  수요측면에서  물가상승압력이  크지  않을 것으로  전망하고  있음을  밝혔다.  즉,  수요측면의  물가상승  압력이  가시화  되지  않는다면  금리인상으로  선회할  가능성이  낮음을  의미한다.  한편,  가계부채와  부동산  가격  상승에  향후  민감하게  대응할  가능성을  보였다.  즉,  가계부채 증가  억제  정책에도  불구하고  서울을  축으로  한  주택가격  상승이  물가상승률을  크게  상회할  경우  금리인상으로  정책방향을  전환할  수  있을  것으로  판단된다.  따라서,  하반기  물가의  경우  CPI보다  주택가격  상승  폭과  지속성에  대한 점점이  요구된다.  주택가격  상승과  실업률  하락이  중첩될  경우  기준금리  인상에  나설  것으로  보기  때문이다.  13개월  연속  1.25%  유지 \uf09f  한은은  세계경제의  회복세가  확대되고  있으나  국내  소비  증가세가  여전히  미흡하다는  인식을  보임.  그러나,  수출과  투자가  개선되면서  견실한  성장세를  이어갈  것으로  기대함.  이와  같은  성장에도  불구하고  인플레이션  압력은  크지  않을 것으로  전망함.  이를  기반으로  기준금리  동결을  통해  통화정책  완화기조를  유지하면서  물가  상승률이  목표수준에서  안정될  수  있도록  정책을  운용해  나갈  것임을  밝힘   세계경제  회복세  확대에  따른  수출과  투자  개선.  반면,  소비는  여전히  미흡 \uf09f  글로벌  주요국의  경제  회복을  반영해  

In [120]:
checklist.remove('케이프투자증권')
checklist

['교보증권',
 '하나증권',
 '키움증권',
 '한화투자증권',
 '신한투자증권',
 '메리츠증권',
 '하이투자증권',
 'DS투자증권',
 '대신증권',
 '미래에셋증권',
 '현대차증권',
 'IBK투자증권',
 'DB금융투자']

##### DB금융투자

In [ ]:
df[df['증권사']=='DB금융투자']

,증권사,날짜,텍스트
5275,대우증권,10.01.15,Daewoo Fixed Income Daily Daewoo 2010. 1. ...
5277,대우증권,10.01.14,Daewoo Fixed Income Daily Daewoo 2010. 1. ...
5279,대우증권,10.01.13,Daewoo Fixed Income Daily Daewoo 2010. 1. ...
5281,대우증권,10.01.12,Daewoo Fixed Income Daily Daewoo 2010. 1. ...
5283,대우증권,10.01.11,Daewoo Fixed Income Daily Daewoo 2010. 1. ...
...,...,...,...
5725,대우증권,08.11.27,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...
5726,대우증권,08.11.26,채권메모 - 2008. 11. 26. 수 서철수 02-768-3501 cybersc...
5727,대우증권,08.11.26,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...
5728,대우증권,08.11.25,Daewoo Bond Brief Daewoo 채권 윤일광(02-768-305...


In [121]:
df[df['증권사']=='DB금융투자'].iloc[0,2]

"DB Financial Investment \x0cINDEX 2020년 채권과 환율 전망 헣 수급과 양으로 바라보자 자산의 가격도 수요와 공급의 법칙을 따를 것이다. 흔해지는 것은 싸지고 희소한 것은 비싸진다. 그런 면에서 2020년의 채권과 환율 시장을 모두 커버하는 핵심 키워드는 물량이 되지 않을까 한다.  채권시장에서는 공급과 수요 측면에서 상당한 변화가 예상된다. 수요 측면에서 특히 장기투자기관의 수요 감소가 두드러진다.   보험사는 1) 현금흐름이 악화되는데다가 2)  FX스왑의 개선으로 해외채권의 매력도가 더 높아질 것이고 3)  파생금융상품을    통한 ALM관리에 나서면서 원화 초장기채 수요가 줄어들 것이다. 국민연금은 중기 자산 배분상 국내외 주식과 대체 투자  z 중심의 자산배분으로 인해서 채권으로의 신규 자금 집행이 거의 ‘0’에 가까울 것이다. 공급 측면도 채권시장에 우호적이지 않다. 정부의 경제 살리기로 인해 과거 흐름과 달리 재정 건전성을 다소 희생하더라도   채권 발행을 크게 늘리는 추세다. 국고채는 올해보다 발행이 30%증가하고 공사들을 통한 자금 조달과 집행도 늘어날 것이다. 전반적으로 이러한 수급 우려는 장기채 영역에 더 부정적이다. 환율도 양적인 지표가 작동한다. 바로 연준의 자산 매입이다. 이는 연준의 부채인 지준을 늘려 글로벌 금융 기관이 보다 더 많은 유동성을 창출할 수 있도록 돕는다. 과거 추세 처럼 지준의 양은 달러와 향후에도 밀접한 관련을 가질 것이다.  DB Financial Investment   \x0cPART 01. PART 02. 채권 수급 한눈에 보기 투자 주체별 채권 수요  1. 2020년 채권 수급: 수요측면  1. FX스왑 개선으로 보험사 초장기채 수요 감소 FX, FI 전략 문홍철 m304050@db-fi.com  2. 2020년 채권 수급: 공급측면  2. 보험사의 현금 흐름이 감소 중  3. IRS를 이용해 ALM관리가 가능  z  4. 연기금의 채권 비중 축소 PART 03. PART 04.

In [123]:
df['텍스트']=df.텍스트.str.split('자료 발간일 현재 본 자료를').str[0]


In [122]:
checklist.remove('DB금융투자')
checklist

['교보증권',
 '하나증권',
 '키움증권',
 '한화투자증권',
 '신한투자증권',
 '메리츠증권',
 '하이투자증권',
 'DS투자증권',
 '대신증권',
 '미래에셋증권',
 '현대차증권',
 'IBK투자증권']

##### 신한투자증권

In [124]:
df[df['증권사']=='신한투자증권']

,증권사,날짜,텍스트
28,신한투자증권,23.08.25,주간 채권 코멘트 신한 Bond.Zip 안재균 연구위원 2023년 ...
30,신한투자증권,23.08.25,국내크레딧 험로를 걷는 여전채 Credit Analyst 정혜진 ☎ 3772...
32,신한투자증권,23.08.24,8월 금통위 성장 하강에 대한 인식 변화 2023년 8월 24일 Fi...
42,신한투자증권,23.08.24,해외채권 금리 결정 요인의 주도권 변화 2023년 8월 23일 Fix...
51,신한투자증권,23.08.22,채권전략 조금씩 다가올 금리 고점 2023년 8월 22일 Fixed In...
...,...,...,...
1645,신한투자증권,22.09.08,신한생각 I 크레딧이슈 2022년 9월 8일 크레딧이슈 ...
1656,신한투자증권,22.09.02,THE BOND IDENTITY 22-09 I 채권전략 2022년 ...
1657,신한투자증권,22.09.02,THE BOND IDENTITY 22-09 I 크레딧전략 2022년 9...
1670,신한투자증권,22.08.29,신한생각 I 채권전략 2022년 8월 29일 해외채권 잭슨...


In [127]:
df[df['증권사']=='신한투자증권'].iloc[0,2]

'주간  채권  코멘트 신한  Bond.Zip     안재균  연구위원 2023년  8월  25일  한국  3Y 한국  10Y 회사채  AA- 회사채  A+ 종가 3.75  ▲ 3.88  ▼ 0.76  ▼ 1.42  ▼ WTD MTD YTD 2.80 ▲  ▲  ▼  ▼ 7.20 ▲ 11.20 ▲  ▼  ▼ 2.90  미국  2Y 14.20  미국  10Y   글로벌  IG   글로벌  HY 종가 5.01  ▲ 4.24  ▼ 1.19  ▼ 3.81  ▼ WTD MTD YTD 6.53 ▲  ▲  ▲  ▲ 15.63 ▲ 28.04 ▲ 6.29 ▼ 3.89 ▼   REVIEW   PREVIEW 9월  국채발행계획  내  기간물별  비중   전년대비  3%대  증가  예상되는  24년  예산안  규모 자료:  기획재정부,  신한투자증권   자료:  기획재정부,  신한투자증권 \uf09e  주간  국고  3년  2.8bp  상승,  4.3%를  상회하던  미국  국채  10년  금리가  4.2%대로  내려오면서  대외  금리  급등세  다소  진정.  8월  금통위에서  향후 성장  하강에  대한  한은의  인식  변화  감지로  주요  국고채  금리  하방  압력  자극.  24년  성장률  전망치  하향은  장기  금리  하락폭을  키우는  요인 \uf09e  성장  하강  감수하며  물가  안정에  총력을  가하겠다는  것이  한은  기존  스탠스.  다만  높아진  중국 경기침체  가능성과  그에  따른  한국  경제  불확실성,  예상에  부합  중인  물가  고려  시  점차  성장 약화에  가중치  두겠다며  인식  변화.  단기적으로 장기물의  상대적  강세를  유도할  수  있는  요인 \uf09e  9월  국채발행계획  규모는  11조원으로  8월  12.9조원  대비  1.9조원  감소.  1~8월  구간별  발행  비중과  비교하면  국고  10년과  국고  20년의  발행 비중  감소  확인.  전체  발행  규모  감소  기대  속 국고  10~

In [126]:
df[df['증권사']=='신한투자증권'].iloc[1,2]

'국내크레딧 험로를  걷는  여전채 Credit Analyst 정혜진 ☎   3772-2667 \uf02a hyejin.j@shinhan.com 여전채  상대적  약세  지속   7월  새마을금고  및  GS건설  이슈  이후  부동산  PF  심리가  위축되며  회사채  대비 여전채의  상대적  약세가  진행되고  있다.  카드채  금리는  7월  들어  AA+  회사채를 상회하며  스프레드  갭이  4.6bp로  확대됐다.  AA-  등급  간  스프레드  격차는  가파르게  확대되며  6월말  12.6bp에서  29.2bp까지  벌어졌다. 크레딧  스프레드  자체는  카드  71.3bp,  AA-  캐피탈  105.6bp로  보합세를  유지하고  있지만,  타  섹터  대비  상대적  약세가  두드러진다.  저하된  펀더멘탈과  부정적 수급  전망을  반영한  추세적인  약세의  신호라는  판단이다. 캐피탈사  전반적으로  이자비용  및  부실채권  조기  인식  과정에서  대손비용의  증가로  수익성  부진이  나타났다.  A급  캐피탈사의  경우  ROA  1.5%로  2020년  코로나19  수준으로  수익성이  저하되었다.  PF  대출  규모를  22년말  8.6조원에서  8.4조원까지  정리하며  자기자본  대비  비율이  하락했으나  여전히  100%에  근접하다. 최근  중국  부동산  리스크  속  글로벌  증시  불안도  우려  요인이다.  주가지수  하락으로  ELS  발행이  감소할  경우  이미  조달  환경이  악화된  여전채  수급에  악재로 작용할  수  있다.  회사채  대비  매력적인  금리  수준임에  이견은  없으나,  금리  움직임에  취약한  여전채  투자에  유의할  시기다. 월간  금리  및  스프레드  변동  요약   채권구분 금리  3Y 스프레드  5Y 3Y 금리  5Y 스프레드  1Y 1Y 1Y 5Y 10Y 1Y 3Y 10Y 10Y 국채  국고채  3.523   3.770   3.821   3.920  특수

In [131]:
df[df['증권사']=='신한투자증권'].iloc[2,2]

'8월  금통위 성장  하강에  대한  인식  변화 2023년  8월  24일  Fixed Income Strategist 8월  금통위,  전체적  긴축  기조  유지했지만  상반기  대비  일부  후퇴 안재균 ☎   3772-1298 \uf02a jk.ahn@shinhan.com 8월에도  기준금리  3.50%  만장일치  동결이다.  관심이  집중됐던  수정경제전망에서 올해  GDP  전망치는  기존  1.4%를  유지했지만,  내년  전망치를  2.2%로  기존  대비 0.1%p  하향  조정했다.  예상보다  부진한  중국  경기와  그  시차를  고려한  결정이다. 물가  전망치는  올해와  내년  모두  기존  3.5%,  2.4%를  그대로  유지했다.  다만  올해  근원물가  전망치를  기존  3.3%에서  3.4%로  상향  조정했다.  여전히  근원물가의  상방  압력이  잔존함을  시사하는  대목이다. 예상대로  이번  금통위에서  7월보다  긴축  기조를  더욱  강화하지  않았다.  오히려 기자회견  내용을  보면  향후  긴축  기조  조정  가능성이  일부  감지된다.  1)  현재의 가계부채  증가에  대한  대응에는  미시적  대응이  우선이라는  점,  2)  미국  통화정책 기조에  비독립적이지만,  시기는  다를  수  있다는  점,  3)  타국  대비  한국  물가는  안정적이란  점에서  근거한다.  총재는  경기  전망의  지속적  하향으로  조금씩  성장으로  무게중심이  이동한다는  점을  크게  부인하지  않았다.  성장  하강을  감수해서라도  물가  안정에  총력하겠다는  기존의  의지와  다소  달라진  모습이다. 우리는  8월  금통위가  전체적으로  긴축  기조를  유지했지만,  상반기  대비  일부  후퇴하기  시작했다고  판단한다.  물론  연준  통화정책  불확실성이  높고,  쉽게  예단할 수  없기에  3.75%  가능성을  계속  열어두고  있다.  그러나  지금의  가계부채  증가세가  

In [ ]:
# 안재균이 좀 개판

In [128]:
df['텍스트']=df.텍스트.str.split('이 자료에 게재된').str[0]


In [129]:
checklist.remove('신한투자증권')
checklist

['교보증권',
 '하나증권',
 '키움증권',
 '한화투자증권',
 '메리츠증권',
 '하이투자증권',
 'DS투자증권',
 '대신증권',
 '미래에셋증권',
 '현대차증권',
 'IBK투자증권']

##### IBK투자증권

In [132]:
df[df['증권사']=='IBK투자증권']

,증권사,날짜,텍스트
1876,IBK투자증권,21.11.08,Executive Summary글로벌펀더멘털점검Post Corona의 경제• Po...
1879,IBK투자증권,21.11.03,ㅣ Weekly ETF ㅣ 2021. 11. 3Global ETF / Quant김인...
1900,IBK투자증권,21.09.29,ㅣ Weekly ETF ㅣ 2021. 9. 29Global ETF / Quant김인...
1906,IBK투자증권,21.09.08,ㅣ Weekly ETF ㅣ 2021. 9. 8Global ETF / Quant김인식...
1919,IBK투자증권,21.07.26,"7월 FOMC 전망: 아직 확인해야할 것들이 남았다  7월 FOMC 회의, 기존..."
...,...,...,...
5453,IBK투자증권,09.09.28,IBKS Bond Daily Brief 국내 채권시장 동향 KOSPI의 조...
5455,IBK투자증권,09.09.25,IBKS Bond Daily Brief Fixed Income Strategist ...
5456,IBK투자증권,09.09.24,IBKS Bond Daily Brief IBK투자증권 리서치센터 국내 채권시장 ...
5462,IBK투자증권,09.09.21,www.ibks.com Fixed Income Strategy 채권분석 2009 년...


In [133]:
df[df['증권사']=='IBK투자증권'].iloc[0,2]

'\x0cExecutive Summary글로벌펀더멘털점검Post Corona의 경제• Post Corona, 글로벌 펀더멘털 점검•2022년 글로벌 경제, 완만한 회복 전망경제의현안인플레이션미국긴축의시작과전망인플레이션과 스태그플레이션• 경제의 현안, 인플레이션• 인플레이션, 지속될 수 있는가?• 인플레이션이 스태그플레이션으로 확장될 가능성긴축의 시작, 새로운 듯 새롭지 않은 길• 미국 테이퍼링 시작, 기준금리 인상의 선반영• 연준이 인플레이션을 대하는 태도• 기대 인플레이션과 실질 금리, 기준금리 인상의 관계한국채권시장전망2022년 한국 경제 및 채권시장 전망• 한국, 수출과 내수 동반 성장 가능• 한국은행의 금융 안정에 대한 강한 의지• 한국 채권시장 및 통화정책 전망\x0cExecutive Summary2022년 한국 채권시장, 상고하저 흐름 예상• 1분기 기준금리 인상 반영과 인플레이션 고점 등으로 금리는 연간 고점 형성할 전망• 2분기 이후 국내 금리는 기준금리 인상 반영 되돌림, 인플레이션 하락 및 하반기 경제지표의완만한 하강 등으로 하락 예상• 2분기 후반 ~ 3분기 초 금리는 미국 테이퍼링 종료 후 기준금리 인상을 앞둔 우려감에 횡보하다가 4분기 소폭 반등 예상• 2022년 한국 기준금리, 1분기 25bp 인상 및 3분기 25bp 인상 전망금리 전망 테이블2021 2022 4Q1Q2Q3Q4Q기준금리국고 3년국고 5년국고 10년미국 기준금리0.00.51.01.52.02.53.021.121.722.122.7기준금리국고3년국고5년국고10년국고20년\x0cExecutive Summary2022년 인플레이션, 1분기 고점 이후 완만한 하락 전망• 현재의 인플레이션을 견인하는 요인들 1) 공급망 차질 2) 고용시장 구인난에 따른 임금 상승3) 국제원자재 가격 등은 내년 경제활동 확대와 경기 회복 등으로 서서히 해소될 전망2022년 미국 채권시장, 2분기 저점 + 4분기 고점의 N자형 흐름 전망• 미국 금리, 올해 4Q~내년 1Q 소폭 상승 후 2분기 접어들면

In [136]:
df[df['증권사']=='IBK투자증권'].iloc[1,2]

'ㅣ Weekly ETF ㅣ 2021. 11. 3Global ETF / Quant김인식02) 6915-5472kds4539@ibks.comETF 자금 흐름 점검과 모멘텀 형성군: 전기차&자율주행✓ 변동성 확대에 대응해 미국 주식 ETF로의 자금 유입세. 업종별 자금 유입은 인플레, 테이퍼링 이슈 반영✓ 기업 실적 기대감은 업종보다 종목 간 차별화로 이어짐 → 테마 ETF 투자로 접근✓ 이익 및 주가 모멘텀 형성 중인 전기차&자율주행 테마 – LIT, DRIV 제시리스크 국면에서 상대적 우위에 있는 미국 주식으로의 자금 유입 확인연준 테이퍼링과 인플레이션 등을 감안한 ETF 자금 유입세가 확인되고 있다. 자산군 기준으로 주식과 채권ETF로는 최근 12주 간 자금 순유입세가 나타났고, 원자재는 세부 자산별 자금 흐름 엇갈린 가운데 박스권등락을 보였다. 안전자산인 채권 ETF의 주간 평균 자금 유입 강도는 0.26%로 경기 둔화 우려, 테이퍼링 이슈를 반영하면서, 타 자산군 대비 우세했다. 주식은 선진국을 대표하는 미국과 신흥국으로 구분해 나타냈을 때, 상대적으로 리스크 국면에서 안정적인 미국 주식으로의 자금 흐름이 나타났다. 원자재 같은 경우 물가 상승 압력 지속에도 선제적 자금 유입에 따라 소규모 자금 유출이 발생한 것으로 보인다. 실적 기대감은 종목 간 차별화로 이어져미국 업종별 ETF 자금 흐름 역시 인플레이션과 시장 금리 상승을 반영한 모습이다. 1개월 누적 자금 유입 강도 상위 업종은 금융, IT, 에너지, 자유소비재이며, 하위 업종은 유틸리티, 필수소비재, 커뮤니케이션서비스순이다. S&P500의 분기 실적 발표는 60% 가량 진행됐으며, 호실적 기대감에 미국 3대 증시는 사상 최고치를 경신 중이다. 하지만 알파벳, 넷플릭스 등의 호실적에도 불구하고 커뮤니케이션서비스업종은 최근 1개월 수익률이 0.2%에 불과했다. 어닝 서프라이즈 발표 기업들에 대한 기대감이 업종 전반으로 이어지기 보다는 종목 간 차별화에 그쳤다는 판단이다. 테마 ETF 비교와 주도 테마

In [135]:
df['텍스트']=df.텍스트.str.split('본 조사분석자료는').str[0]

In [137]:
checklist.remove('IBK투자증권')
checklist

['교보증권',
 '하나증권',
 '키움증권',
 '한화투자증권',
 '메리츠증권',
 '하이투자증권',
 'DS투자증권',
 '대신증권',
 '미래에셋증권',
 '현대차증권']

##### 메리츠증권

In [138]:
df[df['증권사']=='메리츠증권']

,증권사,날짜,텍스트
31,메리츠증권,23.08.25,2023년 8월 24일 Strategy Idea Strategy Idea 미국금...
75,메리츠증권,23.08.17,2023년 8월 16일 Strategy Idea Strategy Idea 주인 ...
147,메리츠증권,23.08.04,2023년 8월 3일 Strategy Idea Strategy Idea Risk...
243,메리츠증권,23.07.21,2023년 7월 20일 Strategy Idea Strategy Idea 누적된...
298,메리츠증권,23.07.13,2023년 7월 12일 Strategy Idea Strategy Idea Inf...
474,메리츠증권,23.06.13,2023년 6월 8일 Strategy Idea Strategy Idea ▲ 해외...
502,메리츠증권,23.06.08,2023년 6월 7일 Strategy Idea Strategy Idea ▲ 채권...
783,메리츠증권,23.04.20,2023년 4월 19일 Strategy Idea Strategy Idea 금리와...
884,메리츠증권,23.04.05,2023년 4월 4일 Strategy Idea Strategy Idea 잠재된 ...
976,메리츠증권,23.03.21,2023년 3월 16일 Strategy Idea Strategy Idea 중요한...


In [139]:
df[df['증권사']=='메리츠증권'].iloc[0,2]

"2023년  8월  24일 Strategy Idea Strategy Idea 미국금리  급등,  진단과  전망  예상보다  강한  Fiscal Impact▲ 채권/자산배분Analyst  윤여삼 yeosam.yoon@meritz.co.kr \uf0fc 미국채10년  지난해  4.3%  고점을  넘어서  2008년  금융위기  이후  최고치를  기록  중\uf0fc 통화긴축  마무리와  함께  안정될  것으로  기대했던  미국  금리경로,  올해  2분기  바닥으로  다시상승추세  전환.  양호한  경기여건과  늘어난  미국채  공급의  이면에  급증한  재정적자  효과  있어\uf0fc 미국  재정이슈는  정치적  쟁점과  부채&이자  부담  점검  필요한  영역  진입.  다만,  단기채  중심발행에  따른  듀레이션  확대  아니라면  채권시장  부담  압력  점차  완화될  것미국채10년  하향안정이  아닌  전고점을  넘어  상승추세로  전환 미국채10년  금리는 지난  해 10월  고점인  4.3% 돌파 다시 보기  힘들 것이라고 생각했던 지난해  10월  고점인 미국채10년  장중 4.3%를 넘어섰다. 2분기  3.3%  정도에서  바닥을  확인한  10년  금리가  여름까지의  반등여력을  인정해도  우리가 예상했던 상단은  3.8%  정도였다.   미국금리 급등을  두고 시장의 표면적  평가는 1)예상보다 양호한  경기, 2)인플레이션에  대한 높은  불확실성,  3)미국채  공급증대에 따른  수급불안  등을 거론한다.  작년의 물가주도  금리상승과는  달리  종합적인  분석이 필요해졌다.   더  높은  수준의 미국채  금리가 이어질 것이라는  전망도  제기 최근  금리가  급등하자 10년  금리는 4%  중반,  아니  5%대,  심지어 6%까지 오를 수  있다는 전망까지 나오고 있다.  금리상승 위험을 경계해온 부분이  있지만, 2분기까지는 찾아보기 힘들었던 의견들이다.   그림1   미국금리  급등,  금융위기  이후  미국채  4%대  

In [ ]:
# \uf0fc 이런거 지워야되는디 안지워지네
# 본  조사분석자료는 제3자에게  사전  제공된  사실이 없습니다.  당사는 자료작성일  현재 본  조사분석자료에  언급된 종목의  지분을 1%  이상  보유하고 있지  않습니다. 본  자료를  작성한  애널리스트는  자료작성일  현재  추천  종목과  재산적  이해관계가  없습니다.

In [141]:
checklist.remove('메리츠증권')
checklist

['교보증권',
 '하나증권',
 '키움증권',
 '한화투자증권',
 '하이투자증권',
 'DS투자증권',
 '대신증권',
 '미래에셋증권',
 '현대차증권']

##### 현대차증권

In [142]:
df[df['증권사']=='현대차증권']

,증권사,날짜,텍스트
1379,현대차증권,23.01.02,Credit + Credit 이화진 hlee@hmsec.com Credit Wee...
2491,현대차증권,19.03.11,Credit Report Credit 박진영 CPA 02) 3787-22...
2512,현대차증권,19.01.21,Credit Report Credit 박진영 CPA 02) 3787-22...
2517,현대차증권,19.01.14,Credit Report Credit 박진영 CPA 02) 3787-22...
2528,현대차증권,19.01.02,Credit Report Credit 박진영 CPA 02) 3787-22...
...,...,...,...
5146,현대차증권,10.10.13,Market Issue 자료제공일: 2010년 10월 13일 Bond Market...
5151,현대차증권,10.10.11,Bond Market View BOND MARKET WEEKLY 자료제공일: 2...
5162,현대차증권,10.10.04,Bond Market View BOND MARKET WEEKLY 자료제공일: 2...
5167,현대차증권,10.09.30,Bond Market Monthly 10월 채권시장 전망기준금리 인상 재개에 따른금...


In [143]:
df[df['증권사']=='현대차증권'].iloc[0,2]

'Credit + Credit  이화진 hlee@hmsec.com Credit Weekly 전섹터 스프레드 축소, 등급전망 하향은 증가 발행시장-채안펀드 가동으로 발행 이어져   -  계절적  발행  비수기임에도  불구하고  SK텔레콤에  이어,  차주에도  롯데건설  수요예측이  예정되어 있음. 채안펀드 가동으로 내년 초로 미뤘던 발행이 나오고 있고, 1월 발행을 준비하는 기업들이 다수  거론되고  있음.  포스코 3,500억원,  유암코 700억원, LG유플러스1,500억원, 롯데제과와  롯데건설  각각  1천억원,  2,500억원,  KT1,500억원,  신세계1천억원  등이며,  발행  예정금액대비  최대  2~3배까지  증액을  계획하고  있는  것으로  알려짐. -  은행대출, CP발행  등  조달  다각화를  통한  회사채  발행  대체  수단도  녹록치  않고,  채안펀드  가동과 기관 매수세 유입이 기대되고 있어 회사채 발행에 나서는 것으로 판단됨. 내년 금리인상이 마무리될 것으로 보이지만, 금리 수준은 높게 유지될 가능성이 크고, 하반기 불확실성이 여전히 남아있는 상황임.  -12월 신용평가사의 정기 평가 시즌으로 신용등급 전망 하향이 증가하고 있어, 비우량등급에 대한 신용경계감이 높아지고 있음. 발행시장에서도 우량등급 중심으로 수요가 집중될 것으로 예상됨. 크레딧 전망-  여전채까지 스프레드 축소,  신용등급 전망 하향은 증가   -20일 일본 중앙은행 BOJ의 기습적 통화정책 변경. 단기금리는 -0.1%로 유지하나, 10년물은 0%  유지하기위해 매입상한없이 매입하는 것도 유지. 10년 국채 상한선을 +/-0.25%에서 +/- 0.5%로 올리면서 일본 통화긴축 신호로 해석됨. 달러-엔 급락하고, 원달러 환율도 하락 함.   -크레딧 스프레드는 공사채 축소 폭이 -11bp로 가장 컸고, 은행채 -6bp, AA등급 여전채와 회사채  -7~-8bp 축소 전환되면서 크레딧 온기가 전체적으로 확대되는 모습임. 다만, A등급 여전채와 회사 채는 -3

In [144]:
checklist.remove('현대차증권')
checklist

['교보증권', '하나증권', '키움증권', '한화투자증권', '하이투자증권', 'DS투자증권', '대신증권', '미래에셋증권']

##### 미래에셋증권

In [145]:
df[df['증권사']=='미래에셋증권']

,증권사,날짜,텍스트
1190,미래에셋증권,23.02.09,     ‘’
1277,미래에셋증권,23.01.20,    ”“ ’
1333,미래에셋증권,23.01.11,   “” ’
1457,미래에셋증권,22.12.07,    “”
1524,미래에셋증권,22.11.09,    “”
...,...,...,...
3104,미래에셋증권,17.04.17,채권분석 윤여삼 yeosam.yoon@miraeasset.com 김민형 ...
3109,미래에셋증권,17.04.14,크레딧 투자전략 이경록 kyoungrok.lee@miraeasset.com...
3110,미래에셋증권,17.04.14,Issue Comment 윤여삼 yeosam.yoon@miraeasset.c...
3111,미래에셋증권,17.04.14,Issue Comment 김민형 minhyung.kim@miraeasset....


In [146]:
df[df['증권사']=='미래에셋증권'].iloc[0,2]

'\uf09f \uf09f \uf09f \uf09f \uf09f   \x0c‘’ \x0c \x0c\x0c\x0c \x0c'

In [148]:
# "텍스트" 칼럼에서 한글이 포함되지 않은 행을 삭제

def contains_hangul(text):
    return bool(re.search('[ㄱ-힣]', text))


df = df[df['텍스트'].apply(contains_hangul)]
df[df['증권사']=='미래에셋증권']

,증권사,날짜,텍스트
1681,미래에셋증권,22.08.24,"Mirae Asset FI Weekly 달러화 강세, 원화 장기채권 매수 ..."
1682,미래에셋증권,22.08.18,Mirae Asset FI Weekly 미국 BEI와 실질금리의 엇갈린 흐름...
1687,미래에셋증권,22.08.11,"Mirae Asset FI Weekly 미국 인플레 안정 조짐, 국내 장기..."
1710,미래에셋증권,22.07.13,Mirae Asset FI Weekly 긴축 부담에도 금리 안정을 기대하는 ...
1746,미래에셋증권,22.05.18,Mirae Asset FI Weekly 아직 안심하기 어려운 국내 긴축 우...
...,...,...,...
3104,미래에셋증권,17.04.17,채권분석 윤여삼 yeosam.yoon@miraeasset.com 김민형 ...
3109,미래에셋증권,17.04.14,크레딧 투자전략 이경록 kyoungrok.lee@miraeasset.com...
3110,미래에셋증권,17.04.14,Issue Comment 윤여삼 yeosam.yoon@miraeasset.c...
3111,미래에셋증권,17.04.14,Issue Comment 김민형 minhyung.kim@miraeasset....


In [149]:
df[df['증권사']=='미래에셋증권'].iloc[0,2]

'Mirae Asset FI Weekly 달러화  강세,  원화  장기채권  매수  기회로 민지희 jihee.min.a@miraeasset.com FI Check Point Fixed Income Market Issue Strategy 달러화 강세와 미국 10년 BEI 상승하면서 국채 10년 금리도 상승  달러화지수 연 고점인 108pt을 상회, 미국 10년 금리도 다시 3%선 돌파  미국  인플레  안정  조짐에도  연준  인사들의  매파적  발언으로  내년  연준의  정책  전환기대감 과도했다는 인식 반영되며 9월 FOMC에서 추가 자이언트스텝 기대감 높아짐  미국  인플레  레벨  아직  높고  양호한  고용지표  등을  고려하면  내년에도  연준의  긴축기조가 유지될 것이라는 우려 지속될 수 있음 연준의 긴축 기조가 쉽게 전환되지 않을 수 있다는 기대감  연준  인사들,  인플레가  정책  목표인  2%  수준까지  낮아질  때까지  금리인상  기조를지속할 필요가 있음을 주장  미국  핵심  PCE  인플레가  월간  +0.3%로  상승할  경우  내년  상반기까지  3%  중반으로낮아지겠으나,  중고차  및  운송비  등  하락세와  글로벌  경기  둔화  흐름  고려하면  월간0.3% 상승률을 하회하는 속도로 예상보다 더 빠르게 안정될 가능성도 높아 보임  8월  잭슨홀  미팅,  파월  역시  지표에  기반한  정책  결정  언급할  것으로  전망.  미국  10년BEI 추가 하락 제한 되겠으나 실질금리 상승세는 유지되지 못할 것으로 판단 미국 경기 및 인플레 경계심 이어지겠으나 국내 장기금리 상승세 지속되기 어려울 것  국내  대중국  수출  감소세.  내년까지  수출  둔화  흐름  이어지고  금통위의  매파적  스탠스꾸준히 완화될 것으로 전망. 연말 국내 기준금리 2.75% 전망 유지  최근  기재부의  내년도  건전  재정  기조  계획은  국채  수급에  우호적이지만,  내년도  국내경기 여건 고려 시 현실화되기 어렵다는 판단. 국내 장기

In [150]:
checklist.remove('미래에셋증권')
checklist

['교보증권', '하나증권', '키움증권', '한화투자증권', '하이투자증권', 'DS투자증권', '대신증권']

##### 대신증권

In [151]:
df[df['증권사']=='대신증권']

,증권사,날짜,텍스트
547,대신증권,23.05.31,2023년 하반기 경제 및 금융시장 전망채권전략인하로 가는 길공동락 dongrak....
1501,대신증권,22.11.18,Fixed Income Economist & Fixed Income Strategi...
3755,대신증권,15.11.23,문남중 Cross Asset Strategist 02.769.3050 ...
3765,대신증권,15.11.16,Sector Strategy PVLT 주간 업종 선호도 Technical An...
3766,대신증권,15.11.16,문남중 Cross Asset Strategist 02.769.3050 ...
3830,대신증권,15.10.12,Global Fund flow Weekly Asset Market Brief 문...
3991,대신증권,15.06.22,Global Fund flow Weekly Asset Market Brief 문...
4025,대신증권,15.02.16,Fundflows 문남중 Cross Asset Strategist njmoon@da...
4038,대신증권,15.02.04,NO. 21 Monthly 2015 Feb. Daishin 2월 펀드시장 전망 ...
4044,대신증권,15.01.30,Fundflows 문남중 Cross Asset Strategist njmoon@da...


In [152]:
df[df['증권사']=='대신증권'].iloc[0,2]

'2023년 하반기 경제 및 금융시장 전망채권전략인하로 가는 길공동락 dongrak.kong@daishin.com\x0c23년 하반기 전망Investment Summary높아진 금리가 이제는 기회로\uf0a7 이례적인 기준금리 인상의 누적으로 경기 모멘텀 둔화, 금융 여건의 악화 등을 반영해 하반기 시중금리의 추세적인 하락 경로를 예상. 또한 기준금리 인하에 대한 기대가 지속적으로 불거짐에 따라시중금리는 레벨 다운 과정을 반복할 것\uf0a7 경기 논쟁은 SVB 사태 이후 하강 또는 침체 쪽으로 확연하게 변화하고 있음. 동시에 금융 불안위험으로 인해 기존 물가 안정에 집중됐던 통화당국의 정책 방향도 이동 중\uf0a7 경기 모멘텀의 뚜렷한 하강 징후가 확인될 경우 기준금리 인하 기대는 상당한 설득력을 확보할수 있을 것. 다만 여전히 절대적으로 높은 물가, 완화적 금융 여건에 대한 부담 등으로 2023년연내 기준금리 인하는 한국과 미국 모두 어려울 전망\uf0a7 실현 가능성과 무관하게 기준금리 인하 기대가 차츰 구체화될 것으로 보이는 만큼 추세적인 채권매수 및 포지션 확대를 권고. 기준금리 인하를 둘러싼 논쟁과 통화당국의 견제가 상당 기간 이어질수 있는데 이를 채권 매수 기회로 활용할 것으로 제안\uf0a7 크레딧 채권의 경우 경기 하강 위험 등을 반영하여 우량물 중심으로 매수를 권고하며 크레딧 스프레드의 등급별 차별화 가능성을 전망TB 10년 금리와 MOVE 지수한국 기준금리와 국고 10년 금리 경로 및 전망자료: Bloomberg, 대신증권 Research Center자료: 한국은행, 금융투자협회, 대신증권 Research Center012345202122232425국고10년4.63%기준금리3.50%60801001201401601802002201.52.02.53.03.54.04.522.0122.0422.0722.1023.0123.04TB 10년금리MOVE\x0c23년 하반기 전망SVB 사태가 몰고 온 변화금융 우위룰 다시 생각하다\uf0a7 실리콘밸리은행 

In [155]:
df[df['증권사']=='대신증권'].iloc[4,2]

'문남중  Cross Asset Strategist    02.769.3050    njmoon@daishin.com 채권선호를 낮추는 美 \uf09e  글로벌 채권형 펀드 플로우: 선진국 순유출 전환, 신흥국 순유출 증가   \uf09e  글로벌 주식형 펀드 플로우: 선진국 순유입 증가, 신흥국 순유출 증가  \uf09e  글로벌 ETF 플로우: 선진국 순유입 감소, 신흥국 순유출 증가                     \x0cGlobal Fund Flow    Weekly   글로벌 펀드 자금 동향  채권형 자금유출 전환, 주식11월  2주차  글로벌  펀드  자금은  채권형이  유출로  전환된  가운데,  주식형형 자금 유입세 지속. 다만, 을  중심으로  유입세를  보였던  한  주  였다.  미국의  10월  고용지표  호조가  12월 12월 미 금리인상 가능성이 금리인상  가능성을  높임에  따라  채권형  선호가  감소한  가운데,  상대적으로  주식형을  중신흥국 권역의 주식형 및 심으로  자금유입이  지속되는  모습이다.  하지만,  주식형  유입규모가  감소하고  있어  위험선채권형 자금선호를 낮추는 모습 호  강도는  낮아지는  모습이며,  채권형  및  주식형의  신흥국  자금유출  압력은  커지는  모습이다.  주식형내  중국도  정책기대감이  지속되고는  있지만  미  통화정책  영향력에서  벗어나지  못하며  2주째  자금유출  흐름을  보이면서  전체  신흥국  자금  유출을  자극하는  모습이다.  다음주도  월간  및  주간  미  고용지표  호조  영향세가  이어지며  미  금리인상  우려가  투자심리를  위축시킬  것으로  보여,  채권형  선호가  낮아지는  가운데  신흥국내  주식형  및  채권형  모두  자금유출  압력이  높아질 것으로  보인다.   그림 3. 글로벌 펀드 누적 플로우 추이  Total 주식형펀드채권형펀드자료: Bloomberg, 대신경제연구소 그림 1.  주식형 펀드 권역별 누적 추이    그림 2

In [154]:
df[df['증권사']=='대신증권'].iloc[-3,2]

'Fundflows 김훈길 hgkim@deri.co.kr   Weekly Asset   Market Brief - 글로벌 채권형 펀드 플로우 방향성 설정 구간 진입 - 글로벌 주식형 펀드 플로우 선진국에서 다시 자산 유출 - 글로벌 ETFs 플로우 유입 규모 축소 ows Economic 2014. 06. 27 www.deri.co.kr                                                                   \x0cGlobal Fund Flows Spot   6 월  4 주  글로벌  채권  : 방향성 설정 구간 진입   선진국 지역으로 유입 재개, 신흥국에 안정적 유입 지속   6월  4주  글로벌  채권형  펀드로는  $4.7bil의  자산이  순유입되어  지난  주  순유출  전환  이후  1주  만에  다시  투자자는  시장으로  복귀했다.  선진국에는  $4.2bil,  신흥국에는  $0.5bil이  각각 유입되었으며 유입규모는 최근 4주 최대규모에 해당한다.  하지만  현  시점은  채권형  펀드  플로우의  방향성을  제시하기가  몹시  곤란한  구간으로  미국 통화정책  결정권자들의  일관적이지  못한  인플레이션  관측이  결국  어떤  정책  스탠스로  귀결될  것인지에  따라  시장의  구조는  완전히  달라지게  될  것이다.  미국채  금리는  6월  이후  상승과 하락이 겹치며 방향성 탐색 구간에 진입했음을 보여주고 있다.  현재 투자자들 역시 견고한 매매전략이 수립되어 있지는 않는 것으로 보이며 시장의 미세한 변동에 따라 언제든 큰 폭의 진입 혹은 진출이 가능할 것으로 보인다. 따라서 당분간은 시장판단보다는 면밀한 모니터링이 우선적으로 필요하다고 판단된다.  표 1. 글로벌 bond fund 주간 자산 이동 asset class division region Developed Emerging North America Global Western Europe Asia pacific sub tot

In [156]:
checklist.remove('대신증권')
checklist

['교보증권', '하나증권', '키움증권', '한화투자증권', '하이투자증권', 'DS투자증권']

##### DS투자증권

In [161]:
df[df['증권사']=='DS투자증권']

,증권사,날짜,텍스트
181,DS투자증권,23.07.28,FOMC Comment FOMC Review 강승연 채권·신용분석 sykang@ds...
259,DS투자증권,23.07.19,Bond Strategy 강력 본드 강승연 채권·신용분석 02-709-2657 sy...
286,DS투자증권,23.07.14,Bond Strategy 7월 금통위 Review 강승연 채권·신용분석 02-709...
332,DS투자증권,23.07.07,Bond Strategy 강력 본드 강승연 채권·신용분석 02-709-2657 sy...
371,DS투자증권,23.06.30,Bond Strategy 강력 본드 강승연 채권·신용분석 02-709-2657 sy...
430,DS투자증권,23.06.20,Bond Strategy 강력 본드 강승연 채권·신용분석 02-709-2657 sy...
493,DS투자증권,23.06.09,Bond Strategy 강력 본드 강승연 채권·신용분석 02-709-2657 sy...
521,DS투자증권,23.06.05,2023 하반기전망 채권 – 방향성은 결국 하락 방향성은 결국 하...
540,DS투자증권,23.06.01,Bond Strategy 강력 본드 강승연 채권·신용분석 02-709-2657 sy...
572,DS투자증권,23.05.26,Bond Strategy 강력 본드 강승연 채권·신용분석 02-709-2657 sy...


In [162]:
df[df['증권사']=='DS투자증권'].iloc[0,2]

'FOMC Comment FOMC Review 강승연 채권·신용분석 sykang@ds-sec.co.kr 데이터 디펜던트라면? 통화정책 결정 : 특별할 것 없던 25bp 금리 인상 7월 FOMC는 기준금리를 5.25~5.50%로 25bp 인상했다. 만장일치의 결과였으며 시장 예상에 부합한 결과였다. 타이트한 고용시장과 완화되고 있지만 여전히 높은 물가가 금리 인상 배경으로 작용했다. 대차대조표 축소 규모는 월간 최대 950억 달러 축소로 변함없이 유지했다.  7월 FOMC 성명서는 6월 성명서와 상당부분 유사했다. 지난번과 동일하게 데이터와 통화정책 파급효과에 근거해서 금리결정을 시행해 나갈 것임을 강조한 것이다. 다만 경기 판단에 대해서는 6월 ‘modest’에서 7월 ‘moderate’로 소폭 상향조정했다.  데이터 디펜던트라면? 기자회견의 핵심 역시 데이터 디펜던트였다. 8월에는 잭슨홀 미팅이 있기 때문에 다음 통화정책 결정 회의까지는 약 8주간의 시간이 남아있다. 파월 의장은 추가적인 금리 인상은 이 기간 축적되는 데이터에 달려 있음을 재차 강조했다.  인플레이션에 대한 유연한 대응을 위해서라도 연준이 추가 금리 인상의 가능성을 열어두는 태도는 상당 기간 지속될 수 있다. 완화적인 스탠스로의 선회는 기대인플레이션 반등으로 나타날 수 있기 때문이다. 자기충족적 성격을 지닌 기대인플레이션을 고정시키기 위해서라도 연준의 Hawkish 기조는 불가피하다. 다만 역설적으로 데이터 디펜던트를 강조했다는 점은 연내 추가 인상의 가능성이 낮음을 의미한다고 판단한다. 최근 물가 상승률 둔화가 확인되며 실질기준금리가 플러스 영역으로 돌아섰다. 이런 상황에서 누적된 가파른 금리 인상은 미국 기업 파산건수가 급증으로 나타나고 있다. 실질금리가 로 진입한 상황에서 시차를 두고 긴축의 효과가 나타날 것이라 예상한다.  연준은 추가적인 금리 인상을 하기 보다는 긴축적 여건을 유지함으로써 물가를 안정시키고 경기 연착륙을 도모하고자 할 것이다. 7월 FOMC에서도 데이터에 대한 강조가

In [163]:
df[df['증권사']=='DS투자증권'].iloc[4,2]

'Bond Strategy 강력 본드 강승연 채권·신용분석 02-709-2657 sykang@ds-sec.co.kr 한국 통화정책의 방향성은? 차별화되는 통화정책 – 핵심은 물가 가팔랐던 통화긴축 사이클의 막바지에 접어들며 미국의 금리 인상을 기계적으로 따라가던 주요국 중앙은행들의 행보가 달라지고 있다. 통화정책의 디커플링이 발생하는 핵심 요인은 ‘물가’이다. 금리를 동결했다가 인상한 국가와 금리인상의 폭을 확대한 국가는 공통적으로 물가가 반등했다는 특징을 지닌다. 반면 금리 동결을 이어가거나 금리 인하를 단행한 국가들은 공통적으로 물가상승세 둔화가 뚜렷하게 나타났다. 물가가 안정된 나라부터 글로벌 통화정책 전환이 시작되고 있는 것이다. 신흥국부터 시작될 통화정책 전환 향후에도 신흥국에서의 물가 안정세는 이어질 것이라고 판단한다. 지난해 인플레이션의 주요인이던 공급 교란이 상당 부분 해소되었으며 신흥국의 물가에서 큰 비중을 차지하는 식품 가격의 하향 안정세 역시 유지되고 있다. 글로벌 기축통화국인 미국의 긴축 재가속화가 없다는 가정하에 하반기 물가가 안정된 이머징 국가들을 시작으로 글로벌 통화 정책 전환이 나타날 것이라고 전망한다.  개별 국가의 펀더멘털에 주목한 채권 투자 권고 각 국가의 국채 금리가 대외 요인에 연동되는 것은 불가피하다. 다만 금리인상 사이클의 마무리 국면에 접어들었다는 점을 감안할 때 하반기에는 각 국가의 펀더멘털 요인에 좀 더 집중할 필요가 있다고 판단한다. 이런 관점에서 한국은 추가 인상의 가능성이 낮은 나라에 속한다고 판단한다. 한국의 물가상승률은 통화긴축을 재가속화한 국가들 대비 안정적인 둔화흐름을 지속하고 있다. 이런 상황에서 하반기 경제정책방향 발표를 앞두고 정부 인사들은 하반기 경제정책의 기조가 물가안정에서 경기부양으로 전환될 가능성을 시사했다. 이런 점을 감안할 때 한국의 통화정책은 물가 안정 후 경기에 집중하는 이머징 국가들과 유사하게 펼쳐질 가능성이 높다고 판단한다. 대외 금리 상승에 연동하며 국고금리가 상승한 현 시점을

In [165]:
checklist.remove('DS투자증권')
checklist

['교보증권', '하나증권', '키움증권', '한화투자증권', '하이투자증권']

##### 하이투자증권

In [166]:
df[df['증권사']=='하이투자증권']

,증권사,날짜,텍스트
34,하이투자증권,23.08.24,올해 성장률을 왜 내리지 않았을까 김명실 msbond@hi-ib.com...
84,하이투자증권,23.08.16,"금리 디커플링, 얼마 남지 않았다 김명실 msbond@hi-ib.com ..."
114,하이투자증권,23.08.09,스팁은 짧고 플랫은 길다 김명실 msbond@hi-ib.com 국고 ...
157,하이투자증권,23.08.03,미국 연착륙의 조건 김명실 msbond@hi-ib.com ◼ Key ...
206,하이투자증권,23.07.27,9월 인상과 동결 모두 오픈 김명실 msbond@hi-ib.com ...
258,하이투자증권,23.07.19,국고채 수급 특징 점검: 약해질 이유는 없다 김명실 msbond@hi...
288,하이투자증권,23.07.13,정책여력 비축 차원의 동결 김명실 msbond@hi-ib.com ◼ ...
299,하이투자증권,23.07.12,생각해야 하는게 많아진 7 월 금통위 김명실 msbond@hi-ib.co...
369,하이투자증권,23.06.30,엔저가 YCC 를 움직일 수 있을까 김명실 msbond@hi-ib.com...
459,하이투자증권,23.06.15,동결 후 인상은 강력한 근거가 필요 김명실 msbond@hi-ib.c...


In [171]:
df[df['증권사']=='하이투자증권'].iloc[0,2]

' 올해  성장률을  왜  내리지  않았을까   김명실 msbond@hi-ib.com ◼  Key takeaways ✓  8 월  금통위,  한은  3.50%  만장일치  동결.  올해  성장률  전망  유지,  내년은  하향 ✓  근원물가에  대한  경계감은  지속.  연내  동결의  주요  배경으로  작용할  것 ✓  내년  성장률  2 회  연속  하향,  채권시장  관심은  통화정책→펀더멘털로  이동 ◼  Key Chart 8 월  통화정책방향문: 국내 경기에  대한  판단, 7 월  대비  다소 부정적으로 변화 자료: 한국은행,  '

In [170]:
df[df['증권사']=='하이투자증권'].iloc[4,2]

' 9월  인상과  동결  모두  오픈   김명실 msbond@hi-ib.com   ◼  Key takeaways ✓  7 월  FOMC, 25bp  만장일치  인상  결정.  성명서  내용은  6 월과  거의  동일 ✓  파월  기자회견, 9 월  인상과  동결  가능성을  모두  오픈 ✓  9 월  추가  인상  유무도  중요하나  오히려  금융시장에서는  고점을  얼마만큼 유지할  수  있을지  여부에  더  관심이  쏠릴  것. 7~9 월  지표  결과에  따라  정책 불확실성이  재확대  될  경우  미국채  10 년, 4.08%까지는  추가상승  여력  유효 9 월,  인상과  동결  가능성을  모두  열었다 연준은  7 월  FOMC 에서  정책금리인  연방기금금리  목표치를  기존  5.00~5.25%에서  25bp 인상한  5.25~5.50%로  상향  조정함 성명서와  기자회견  내용을  종합했을  때  FOMC 의  핵심  내용은  크게  3 가지로  요약 1)  성명서,  경기  평가  부문을  제외하고는  지난  6 월  FOMC  내용과  거의  동일.  인상 종료에  대한  신호는  부재 :  최근  지표들은  경제활동이  ‘보통의’  속도로  확장되고  있음을  시사한다고 언급함.  이는  기존에  ‘완만한’  속도로  계속  확장되고  있음을  시사한다는  표현에서 경기  평가를  상향한  것 그  외  연준  위원회는  통화  정책을  위해  추가적인  정보와  영향을  계속  평가할  것이라는 기존  의견을  고수.  연준이  향후  지표에  따라  정책을  결정하겠다는  의미로,  추가  인상 가능성은  여전히  열어  둔  것으로  해석 2)  파월  의장의  기자회견,  9 월  인상과  동결의  가능성  공존.  9 월  인상과  함께  동결 가능성도  함께  언급되며  성명서와  달리  연준  커뮤니케이션은  직전  6 월  대비  매파적  톤이 약화된  것으로  추정.  다만  더  중요한  점은  연준의  추가  

In [172]:
df[df['증권사']=='하이투자증권'].iloc[-3,2]

' 채권사이클이  돌아온다       김명실 msbond@hi-ib.com /  류진이 jinlee@hi-ib.com ◼  Key takeaways ✓  채권형  상품  투자의  근본은  ‘신용위험’에서  시작한다 ✓  신용스프레드와  수익률곡선이  만드는  불편함은  채권  강세장을  만들어  낸다 ✓  망가진  수익률이  걱정되나  채권형  상품  롱  베팅  시점을  구상해야  할  시기다 ◼  Key Chart 신용스프레드와  수익률곡선으로  보는  경기사이클:  엉킨  실타래를  풀어볼  때가  왔다   자료: Bloomberg,  하이투자증권  리서치센터 채권형  상품  투자의  근본은  ‘신용위험’에서  시작한다 미국  하이일드  OAS 가  연초대비  250bp  확대되며  522bp 에  도달했다.  국내도 상황은  크게  다르지  않다.  국채  대비  회사채  AA-  3 년  스프레드는 116.3bp로  연초  60.8bp  대비  50bp  넘게  확대됐다.  투기등급  상황은 더  좋지  않다.  BBB-  3 년  스프레드는  701.7bp로  연초  646bp  대비 55bp  확대됐다.  과거  금융위기  당시  최대  895bp 까지  벌어졌던  상황과  비교해 보면  최악의  케이스에  도달할  시점이  얼마  남지  않은  것으로  보인다. '

In [173]:
checklist.remove('하이투자증권')
checklist

['교보증권', '하나증권', '키움증권', '한화투자증권']

##### 한화투자증권

In [174]:
df[df['증권사']=='한화투자증권']

,증권사,날짜,텍스트
7,한화투자증권,23.08.29,주간 크레딧 동향 카드사발 채무조정 신청 건수 증가 상반기 채무조정 신청 건...
12,한화투자증권,23.08.28,2023년 8월 28일 Hanwha Fixed Income 한화 FI Weekly ...
21,한화투자증권,23.08.25,2023년 8월 25일 Hanwha Fixed Income 한화 Fed Monito...
35,한화투자증권,23.08.24,2023년 8월 24일 Hanwha Fixed Income 한화 BOK Review...
43,한화투자증권,23.08.23,2023년 8월 23일 Hanwha Fixed Income 한화 BOK Previe...
...,...,...,...
5114,한화투자증권,10.11.08,2011년 채권시장전망 유동성은 두 얼굴을가진 헐크다 투자분석팀ᅵ리서치본부 ...
5131,한화투자증권,10.10.25,2010. 10. 25 │ 투자분석팀 BOND BRIEF Analyst 박태근...
5135,한화투자증권,10.10.20,www.koreastock.co.kr 2010. 10. 20 l 코멘트 Bond ...
5144,한화투자증권,10.10.14,www.koreabond.co.kr Bond Issue Comment 10월 금...


In [175]:
df[df['증권사']=='한화투자증권'].iloc[0,2]

'주간  크레딧  동향   카드사발 채무조정 신청 건수 증가 상반기 채무조정 신청 건수는 작년 전체 신청자의 70%에 육박하는 수치다. 채무조정대상자의 현황을 살펴보면 여러 계좌를 통해 대출을 받는 다중채무자가 대부분이었고, 신용카드사를 주 대출기관으로 이용하고 있었다. 신용 스프레드 동향 지난주  신용  스프레드는  보합  수준에  머물렀다.  전반적으로  강보합세를  보였고 여전채는  예외적으로  약보합  마감했다.  공사채  AAA등급  3년물은  0.2bp  하락했고,  은행채  AAA등급  3년물은  0.2bp  하락했다.  회사채  AA등급  3년물  스프레드는 0.2bp 하락했고, 여전채 AA등급 3년물은 0.1~0.3bp 상승했다.  회사채 수요 예측 및 신용등급  발행시장에서는  발행물량이  증가한  가운데  전반적인  강세  분위기를  보였다.  포스코인터내셔널,  연합자산관리,  동원  F&B은  모두  언더발행했다. 반면 BBB0등급을 보유한 이랜드월드는 오버발행했다.  지난  8월  22일  현대로템의  유효등급은  ‘A-/긍정적’에서  ‘A0/안정적’으로  상향됐다.  큰  폭의  수주확대를  통해  매출규모의  증가  추세가  지속될  것으로  예상되기 때문이다.  크레딧 시장 코멘트 채무조정 신청 건수와  변제 기간이 증가한 것은 금리 상승과  경기 둔화가 지속되면서  한계차주의  부실이  가시화되기  시작한  결과로  판단한다.  채무조정자의 주 대출기관이 신용카드사라는 점을 고려하면 카드사 대출의 질적 수준과 건전성 하락에 대한 모니터링이 필요하다는 입장이다. 2023 년 8 월 29 일 크레딧 Weekly ▶Credit Analyst 안소영 So.an.1@hanwha.com  이 자료는 투자자의 증권투자를 돕기 위해 당사 고객에 한하여 배포되는 자료로서 저작권이 당사에 있으며 불법 복제 및 배포를 금합니다. 이 자료에 수록된 내용은 당사 리서치센터가 신뢰할 만한 자료나 정보출처로부터 얻은 것이지만, 당사는 그 정확성이나 완전성

In [176]:
df[df['증권사']=='한화투자증권'].iloc[4,2]

'2023년 8월 23일 Hanwha Fixed Income 한화 BOK Preview 움직일 이유가 없다 ▶ 채권전략 김성수 sungsoo.kim@hanwha.com / 3772-7616  8월 통화정책방향 결정회의: 만장일치 동결 전망 주요 채권금리 추이 24일  금통위에서는  기준금리  만장일치  동결  전망.  향후  3개월  기준금리  전망도  금통위원  전원  3.75%  유지될  것으로  예상.  당분간  기준금리  또는 통화정책 기조에 변화를 줄 유인은 적음 추가 인상이 필요하지 않은 이유 통화정책  시차를  고려하면  지금은  정책  효과가  본격적으로  나오는  시기.  물가  상승률이  8~9월  소폭  반등해도  둔화  추세가  바뀔  가능성은  제한적.  기준금리 레벨도 긴축 영역에 들어선 만큼 긴축 기조 ‘유지’로도 충분 한미  기준금리  역전  폭  추가  확대  여부도  크게  우려할  부분은  아님.  금리  차가 환율, 금융시장에  절대적인  영향을  미치는  요소가  아니라는 것은 이미  증명. 이창용 총재의 “금리 차보다 연준의 스탠스가 중요” 발언에 주목할 필요 연말연초 인하도 필요하지 않은 이유 물가는  기저효과 약화, 원자재  가격  불확실성  등으로  상승세  둔화  속도가 느려질  전망.  2%  중반대  물가  상승률을  확인하기  전까지는  긴축  기조를  유지할 가능성이 높다는 판단. 경기가 좋다고  이야기할 수는  없어도 개선세가 이어지고 있는 것은 사실. 경기 부양 목적 인하 가능성은 사실상 0%에 수렴 금통위원  전원은  가계  부채  문제는  정책  스탠스의  완화가  아닌  강화로  대응해야  한다는  것에  동의.  개인  투자자들의  위험  자산  선호  심리가  사라지지 않았고,  누증된  가계  초과저축을  고려하면  한국은행은  부채를  통한  자금  조달 효용 억제에 집중할 것. 이를 위한 가장 쉬운 방법은 높은 금리의 유지 부동산  PF  등  금융  불안  리스크는  금리가  아닌  유동

In [177]:
df[df['증권사']=='한화투자증권'].iloc[-3,2]

'www.koreastock.co.kr 2010. 10. 20 l  코멘트 Bond Market Comment 중국의  기준금리인상:  직접적인  국내긴축  재료는  아니지만… 국내외  경제ㅣ투자분석팀Analyst 박태근  02)3772-8434 park00@koreastock.co.kr 중국의  기준금리인상 올해  들어  3 차례  지준율  인상  이후  최초의  금리인상  8 월  중국  소비자물가지수가  22 개월  만에  최고치인  3.5%로  치솟고 9 월에는  농산물가격  불안으로  더  높은  수준을  기록할  것으로  우려되고 있는  가운데  낮아진  실질금리와  인플레이션을  잡겠다는  중국  정부의 의지를  나타낸  것으로  판단  현재  중국은행의  1 년  만기  예금금리가  2.25%인  상황에서  실질금리 마이너스인  점을  감안하면  물가상황에  따라  연내  추가  금리인상 가능성, 21 일  발표될  9 월  소비자물가도  3.7%정도의  높은  상승률  예상  중국의  기습금리  인상으로  전일  주요  해외증시는  큰  폭  약세,  미  국채는 중,  장기물  강세,  상품시장은  안전자산  선호에  따른  달러강세로  조정확대  통상적인  과거  중국의  단발성  긴축은  국내  경기  둔화 우려  증시  조정  확대,  채권  중,장기금리  하향엔  우호적  재료,  직접적으로 국내  금리인상  내지  긴축  재료로  반영된  예는  많지  않음.  한편  10 월  국내  소비자물가는  전월비로는  상품물가의  되돌림  가능성이 있어  상승  폭  둔화,  전년  동월비로  4%  내외를  기록할  것으로  예상  국내  채권시장  입장에선  경험상  중국의  긴축이  조기에  국내  금리인상 재료로  반영되는  경우는  많지  않았음.  역시  기대  인플레보다는  긴축시 경기둔화에  기대와  대한  판단이  컸기  때문  반면  최근  채권시장  흐름은  과거대비  경기둔화  우려대비  글로벌  유동성의 유출입  정

In [ ]:
# 얘들은 중간중간 투자자 돕기위해 이자료는 내용을 적어놨는데 위에서 다 삭제해버림.... 쉿!

In [178]:
checklist.remove('한화투자증권')
checklist

['교보증권', '하나증권', '키움증권']

##### 키움증권

In [179]:
df[df['증권사']=='키움증권']

,증권사,날짜,텍스트
5,키움증권,23.08.30,2023 년 08 월 30 일 키움증권 리서치센터 투자전략팀 채권전략 안예하 yha...
16,키움증권,23.08.28,KIWOOM ESG Weekly 키움증권 리서치센터 투자전략팀 채권전략•ESG ...
20,키움증권,23.08.28,2023 년 08 월 28 일 키움증권 리서치센터 투자전략팀 채권전략 안예하 yha...
33,키움증권,23.08.24,2023 년 08 월 24 일 키움증권 리서치센터 투자전략팀 채권전략 안예하 yha...
65,키움증권,23.08.21,2023 년 08 월 21 일 키움증권 리서치센터 투자전략팀 채권전략 안예하 yha...
...,...,...,...
4954,키움증권,11.04.19,크레딧이슈: KB국민카드 KB카드로 이관되는 채권의 스프레드 전망 이화...
4957,키움증권,11.04.18,Weekly Bond MarketWeekly Bond Market 너무 낙관적인 ...
4961,키움증권,11.04.12,Weekly Bond MarkettekraMdnoBylkeeWWeekly Bond ...
4966,키움증권,11.04.05,D-5-3bpD-20-7bp-5bp-24bp-3bp-29bp-2bp-25bp-1bp...


In [180]:
df[df['증권사']=='키움증권'].iloc[0,2]

'2023 년 08 월 30 일 키움증권 리서치센터 투자전략팀 채권전략 안예하 yhahn@kiwoom.com 2024 년  예산안  발표  영향 Check Point •2024 년  정부  예산안  발표.  재정지출  증가율이  20 년  내  최저  수준으로  발표했으며,  재정  긴축  기조가  내년에도 이어질  것이라는  정부의  의지를  확인함.  더  나아가  2024 년  국채  발행  158.8 조  원으로  전년비  9 조원  가량 감소하고,  순증  규모  또한  11.2 조원  감소할  것으로  예상되며  이에  따른  채권시장  공급  부담은  당분간  크게 높아지지는 않을  것으로  판단함 •향후  경기  하강에  따른  추경의  가능성을  완전히  배제할  수  있는  상황은  아니지만,  현재로서  발행량이  계속해서 감소할 것으로 예상되는  점은 커브 스티프닝보다는  플래트닝 기조가  지속될 가능성이  높아보인다.  특히   2023 년 중장기물  발행  비중을  줄였고,  내년에도  비슷한  수준으로  유지될  경우  장기물  수급  부담은  낮게  유지될  것으로 판단함 •이를  감안하면  최근  확대된  3-10  스프레드가  장기화될  가능성은  낮다는  판단임.  단기적으로  한은과  미  연준의 추가  긴축에  대한  경계감이  유지된다는  점을  감안할  때  연말까지  현수준의  스프레드  확대  기조  장기화는 제한적임.   8 월  29 일  정부의  2024 년도  예산안이  발표되었다.  한국  정부는  2024 년  총지출  예산을  656.9 조원으로 2023 년  638.7 조원  대비  2.8%  증가한  수준으로  발표했다.  2024 년  총지출  증가율은  2005 년  이후  20 년내 역대  최저  증가율로,  재정  정상화를  통한  대외  건전성을  확보하겠다는  의지를  재확인했다.  특히  정부는 2023 년  예산안은  건전재정  기조로  전면  전환한  것이며,  2024 년  예산안은

In [181]:
df[df['증권사']=='키움증권'].iloc[4,2]

'2023 년 08 월 21 일 키움증권 리서치센터 투자전략팀 채권전략 안예하 yhahn@kiwoom.com 지난주 동향 지난주  국고채  금리는  상승했다.  미국의  소비지표가  개선세를  보였고,  미  연준  위원의  매파적인  발언  또한  미 연준의  추가  인상  우려가  부각시키면서  국내  시장에도  약세  요인으로  작용했다.  특히  장기채  중심으로  상승 압력을  보이면서  커브는  스티프닝되었다.  국고  3 년물은  전주  대비  4.9bp  상승한  3.726%를  기록하  가운데, 국고  10 년물은  12.5bp  상승한  3.905%에서  거래를  마감했다.  국고  3-10 년  스프레드는  17.9bp 로  전주의 12.5bp  대비 확대되었다.   지난주  채권시장은  중국  경기  불안에  따른  안전자산  선호가  주  후반  강화되기도  했으나,  주  중  전반적으로 미국  추가  인상  우려  등을  반영하면서  약세를  보였다.  특히  미국  소매판매가  전월비  0.7%  상승하면서 예상치를  크게  상회했고,  이에  따라  미국  경기  연착륙  기대가  확대된  것이다.  더  나아가  미니애폴리스 연은  총재도  금리인상이  끝났다고  말하기에는  이르다고  주장하면서  금리  상승  요인으로  작용했다.  특히  미국 소비자물가는  예상을  하회했으나,  PPI 가  시장  예상을  상회하면서  최근  상승한  국제유가를  중심으로  인플레 리스크가  재부각되는  양상을  보였다.  헤드라인  물가  하락의  둔화로  인해  미  연준의  추가  인상  가능성이  부각된 것이다. 더  나아가  FOMC  의사록에서  또한  인플레이션  리스크에  주목할  필요가  있으며,  이에  따라  더  높은  수준의 금리가  필요하다는  입장을  확인했다.  일부  위원들이  과도한  긴축에  대한  경계감이  필요하다는  입장을  보이기도 했으나,  추가  인상에  대한  우려  속에  미국채  10 년물

In [182]:
df[df['증권사']=='키움증권'].iloc[-3,2]

"Weekly Bond MarkettekraMdnoBylkeeWWeekly Bond Market 전격적인  긴축  가능성,  유동성  사정이  달라질  조짐 Fixed Income Strategist  유재호 02) 3787-5283 jhyoo@kiwoom.com 이화진  02) 3787-4711 hjlee@kiwoom.com 안상우  02) 3787-5012 charles@kiwoom.com 이소원  02) 3787-4952 sowonlee@kiwoom.com   ECB, 긴축개시: 포르투갈, 스페인 보다 물가가 걱정이야! ECB가  메이저  중앙은행  중  처음으로  정책금리  인상을  단행했다.  일본-리비아-포르투갈  문제가  경기에  주는  타격  보다,  더  중요한  것은  물가안정임을  인식하고  있다는  증거다.  유로존의  독일  국채금리는  당일  강보합을  기록했으나,  최근  20일간  2년과  10년물이 13~17bp 올라,  금리인상에  대비해  왔다.  포르투갈에는  이제  1천억$가  투입될  예정이고,  스페인의금융시장은  안정을  유지하고  있다.  세계  경기는  별  탈없이  순항중이다.     이번주 하이라이트: 한국은행 기준금리 인상 당사는  4월  금통위에서  기준금리가  인상될  것으로  전망한다.  자세한  근거는  본문의  이슈자료에서  설명했다.  사람들은  예기치  못한  이벤트가  왔을  때,  당황하기  마련이다.  지난  1월  기준금리  인상에  당일  국고  3년~10년  금리는  6~10bp  상승한  바  있다.  당국의  강한  긴축의지는  시장에서  보는  연말  기준금리  예상치를  상향시킬  것인데,  이렇게  되면  기준금리인상을  불확실성  해소로  해석하지  못하게  된다.   4월 주간 금리 전망 4/4 ~ 4/8 실적 4/11 ~ 4/15 전망 국고채 3년 국고채 5년     주:  표안의  숫자는  평균   양호한 수급구조에 변화의 조짐: 실세콜금리 3.04%까지 상승 펀더멘탈이

In [183]:
df[df['증권사']=='키움증권'].iloc[-1,2]

'Morning Bond11/4/1Fixed Income Analyst 유재호  02) 3787-5283  jhyoo@kiwoom.com,  안상우 02) 3787-5012  charles@kiwoom.com, 이소원 02) 3787-4952 sowonlee@kiwoom.comI. 국내 주요 금리국내 금리국고채 1y국고채 3y국고채 5y국고채 10y국고채 20y통안채 91d통안채 1y통안채 2y콜금리 1dCD 91dCP 91d종가D-1-1bp-1bp-1bp-1bp0bp0bp-2bp-1bp0bp0bp2bpD-52bp7bp5bp1bp2bp2bp1bp6bp-4bp0bp3bpD-20-5bp-20bp-23bp-23bp-20bp11bp-7bp-18bp23bp16bp19bp금리 변동폭국고국고국고1y3y5y국고10y국고20y통안91d통안통안회사회사1y2yAA- 3yBBB- 3yMarket Comment국내 시황▪ 31일 국내 채권시장은 미국채 금리 하락, 산업활동 부진, 외인의 대규모 국채선물 순매수로 강세. 단, CPI 발표를 하루 앞두고 금리 하락폭은 제한. 원/달러환율 급락으로 외인 수요 증가 기대와 인플레 완화 기대도 있었으나 외인의 차익실현 매물 출회 우려도 부각.▪ 4월 국고채 6.0조원 발행 계획.▪ 한은 통화정책 보고서: "경제가 견조한 성장을 지속하는 가운데 물가안정기조가 유지되도록 하는 데 중점을 두고 통화정책 운용. 장기간 지속된 완화기조로나타날 수 있는 가계부채 증가, 자산가격 상승 등에 따른 금융·경제의 불균형 심화 가능성에 유의". 김중수 총재: "지금은 인플레이션이 아마 가장 중요한 정책변수일 것. 이를 관리할 한은이 가진 유일한 도구는 바로 금리".▪ 외국인: 국채 +250억원, 통안채 +10억원, 국채선물 +6,156계약. ▪ KTB1106 102.96, 주가 2,106.7pt, 환율 1,096.9원주: 콜금리를 제외한 국내 주요 금리는 금융투자협회 최종호가수익률II. 해외 주요 금리해외 금리미국 TB 2y 미국 TB 5y 미국 TB 10y 미

In [184]:
df[df['증권사']=='키움증권'].iloc[-2,2]

'D-5-3bpD-20-7bp-5bp-24bp-3bp-29bp-2bp-25bp-1bp-21bp-1bp4bp-6bp-11bp-3bp-19bp23bp10bp19bp-1bp0bp6bpD-51bp1bp-1bp-2bpD-20해외 금리6bp 일본 JGB 2y -1bp 일본 JGB 10y -9bp 호주 2y -15bp 호주 10y -11bp-10bp 말레이시아 2y 14bp12bp7bp8bp11bp12bp-1bp 말레이시아 10y -1bp 대만 2y *6bp 대만 10y *10bp 태국 2y 1bp 태국 10y 9bp LIBOR O/N * 대만 채권시장 휴장으로 4/1 종가I. 국내 주요 금리국내 금리국고채 1y국고채 3y국고채 5y국고채 10y국고채 20y통안채 91d통안채 1y통안채 2y콜금리 1dCD 91dCP 91d종가D-10bp1bp-1bp-1bp0bp0bp-1bp0bp-1bp0bp1bp주: 콜금리를 제외한 국내 주요 금리는 금융투자협회 최종호가수익률II. 해외 주요 금리해외 금리미국 TB 2y 미국 TB 5y 미국 TB 10y 미국 TB 30y 미국 TIPS 10y미국 모기지 미국 모기지 유로 Bond 2y 유로 Bond 10y 영국 Gilt 2y 영국 Gilt 10y 종가D-1-4bp-5bp-2bp-1bp-5bp주간주간-1bp-0bp-0bp0bp한국 3월 외환보유고외환보유고 증감외환보유고 잔액Morning Bond11/4/5Fixed Income Analyst 유재호  02) 3787-5283  jhyoo@kiwoom.com,   안상우 02) 3787-5012  charles@kiwoom.com,  이소원 02) 3787-4952 sowonlee@kiwoom.com금리 변동폭국내 시황Market Comment국고국고국고1y3y5y국고10y국고20y통안91d통안통안회사회사1y2yAA- 3yBBB- 3y▪ 4일 국내 채권시장은 금통위 전까지 큰 변동없는 장이 지속될 것이라는 인식으로 박스권 장세 구현. 한편 바이백 수급장이 지속되며 대상채권들은 상대적 강세. 국

In [187]:
# '증권사' 칼럼 값이 '키움증권'인 행의 '텍스트' 칼럼 값을 업데이트
df1=df
mask = df1['증권사'] == '키움증권'
df1.loc[mask, '텍스트'] = df1.loc[mask, '텍스트'].str.split('Compliance Notice').str[0]

df1[df['증권사']=='키움증권'].iloc[0,2]

'2023 년 08 월 30 일 키움증권 리서치센터 투자전략팀 채권전략 안예하 yhahn@kiwoom.com 2024 년  예산안  발표  영향 Check Point •2024 년  정부  예산안  발표.  재정지출  증가율이  20 년  내  최저  수준으로  발표했으며,  재정  긴축  기조가  내년에도 이어질  것이라는  정부의  의지를  확인함.  더  나아가  2024 년  국채  발행  158.8 조  원으로  전년비  9 조원  가량 감소하고,  순증  규모  또한  11.2 조원  감소할  것으로  예상되며  이에  따른  채권시장  공급  부담은  당분간  크게 높아지지는 않을  것으로  판단함 •향후  경기  하강에  따른  추경의  가능성을  완전히  배제할  수  있는  상황은  아니지만,  현재로서  발행량이  계속해서 감소할 것으로 예상되는  점은 커브 스티프닝보다는  플래트닝 기조가  지속될 가능성이  높아보인다.  특히   2023 년 중장기물  발행  비중을  줄였고,  내년에도  비슷한  수준으로  유지될  경우  장기물  수급  부담은  낮게  유지될  것으로 판단함 •이를  감안하면  최근  확대된  3-10  스프레드가  장기화될  가능성은  낮다는  판단임.  단기적으로  한은과  미  연준의 추가  긴축에  대한  경계감이  유지된다는  점을  감안할  때  연말까지  현수준의  스프레드  확대  기조  장기화는 제한적임.   8 월  29 일  정부의  2024 년도  예산안이  발표되었다.  한국  정부는  2024 년  총지출  예산을  656.9 조원으로 2023 년  638.7 조원  대비  2.8%  증가한  수준으로  발표했다.  2024 년  총지출  증가율은  2005 년  이후  20 년내 역대  최저  증가율로,  재정  정상화를  통한  대외  건전성을  확보하겠다는  의지를  재확인했다.  특히  정부는 2023 년  예산안은  건전재정  기조로  전면  전환한  것이며,  2024 년  예산안은

In [188]:
mask = df['증권사'] == '키움증권'
df.loc[mask, '텍스트'] = df.loc[mask, '텍스트'].str.split('Compliance Notice').str[0]

In [185]:
checklist.remove('키움증권')
checklist

['교보증권', '하나증권']

##### 하나증권 날렸습니다..

In [189]:
df[df['증권사']=='하나증권']

,증권사,날짜,텍스트
1,하나증권,23.08.30,2023년 8월 30일 I Global Asset Research Credit...
11,하나증권,23.08.28,잇단 신용 강등에도 은행채 스프레드 하락 자산배분/해외크레딧 W...
24,하나증권,23.08.25,2023년 8월 24일 I Global Asset Research Fixe...
38,하나증권,23.08.24,2023년 8월 24일 I Global Asset Research Glob...
45,하나증권,23.08.23,2023년 8월 23일 I Global Asset Research Credit...
...,...,...,...
3967,하나증권,15.07.14,회사채 수요예측 예정기업 대상 2015년 7월 14일 ★ 기업 및...
3968,하나증권,15.07.14,2015년 7월 14일 BONDSTONE Global Asset Research...
3972,하나증권,15.07.10,회사채 수요예측 예정기업 씨제이헬로비전 2015년 7월 9일 ★ 기...
3973,하나증권,15.07.10,2015년 7월 9일 BONDFLASH Global Asset Research ...


In [191]:
df[df['증권사']=='하나증권'].iloc[0,2]

'2023년  8월  30일  I Global Asset Research Credit CrediVille 아낌없이  주는  은행 Compliance Notice   '

In [192]:
df[df['증권사']=='하나증권'].iloc[4,2]

'2023년  8월  23일  I Global Asset Research Credit CrediVille 유비무환 Compliance Notice   '

In [193]:
df[df['증권사']=='하나증권'].iloc[-3,2]

'회사채  수요예측  예정기업 씨제이헬로비전   2015년  7월  9일   ★  기업  및  발행개요 소속그룹   CJ  주요주주 CJ오쇼핑 53.92% 수요예측일  7.10  발행일 7.17  발행금리 /금리밴드 3Y 700억원/3Y 개별민평  발행목적 운영자금  Credit  손은정 eunjungson@hanafn.com \uf09f CJ계열의 종합유선방송사업자, 우수한 가입자기반과 다양한 사업 보유 ★  민평금리추이 동사는 CJ그룹의 SO로 23개의 SO를 포함한 국내 1위  CATV사업자로 우수한  시장지위를 보유하고 있다. 2012년 1월 KT망 임대를 통해 MVNO사업을 개시하면서 사업분야를  확장했다.  유료방송시장이  성숙기에  진입하면서  성장성은  다소  둔화되었으나 아날로그 가입자의 디지털 전환 등을 통한 매출의 질적 향상을 제고하고 있다. \uf09f 외형성장으로 인한 영업수익률 저하, 재무안정성은 양호  SO인수  및  MVNO  사업  등  외형확대를  통해  이익규모는  증가했으나,  이에  대한 자금소요로  영업수익률은  과거대비 다소  하락한 상황이다. 투자증가에 따라 차입금  규모가  확대되었으나 부채비율 자료:  민평3사,  하나대투증권 ★  발행잔액  만기도래 등 재무안정성 지표는 양호한 상황이며 최근 개선되고 있다. \uf09f 향후 유료방송시장의 경쟁구도 및 MVNO사업 수익에 대한 모니터링 필요 현재  CATV  1위  사업자로  안정적인  시장지위를  유지하고  있으나  통신사업자들의 IPTV나  위성방송과의  경쟁이  불가피한  상황이다.  MVNO서비스는  가입자가  증가하면서 꾸준히 성장하고 있으나 아직 의미있는 수익 기여까지는 시간이 소요될 것으로  보인다.  향후  IPTV와의  경쟁구도에  따른  성과와  MVNO사업의  수익성에  대한 지속적인 모니터링이 필요하다.  자료: KIS-Net,  하나대투증권 SWOT Analysis Strength    Weakness  • 국내 1위 CATV 사업자로 안정

In [194]:
# 하나 증권도 위에 Compliance Notic가 존재.....
df.drop(df[df['증권사']=='하나증권'].index, inplace=True)

In [190]:
checklist.remove('하나증권')
checklist

['교보증권']

##### 교보증권

In [195]:
df[df['증권사']=='교보증권']

,증권사,날짜,텍스트
0,교보증권,23.08.30,Daily 2023.08.30 Fixed Income 백윤민 02_3771_91...
8,교보증권,23.08.29,Daily 2023.08.29 Fixed Income 백윤민 02_3771_91...
13,교보증권,23.08.28,Daily 2023.08.28 Fixed Income 백윤민 02_3771_91...
14,교보증권,23.08.28,Fixed Income 백윤민 yoonmin1983@iprovest.com 202...
15,교보증권,23.08.28,Fixed Income Strategy 발간일자 : 2023/ 08/ 28 백...
...,...,...,...
2867,교보증권,17.11.03,Daily 2017.11.03 Fixed Income 백윤민 02_3771_91...
2870,교보증권,17.11.02,Daily 2017.11.02 Fixed Income 백윤민 02_3771_91...
2874,교보증권,17.11.01,Daily 2017.11.01 Fixed Income 백윤민 02_3771_91...
2875,교보증권,17.10.31,Daily 2017.10.31 Fixed Income 백윤민 02_3771_91...


In [196]:
df[df['증권사']=='교보증권'].iloc[0,2]

"  Daily 2023.08.30 Fixed Income 백윤민 02_3771_9175 / yoonmin1983@iprovest.com Analyst 정윤정 02_3771_9733 / yj.jeong@iprovest.com Market Comment  상하 Today's Key Chart 미국 미국 채권시장, 7 월 채용공고 감소에 따른 고용 둔화 기대로 강세 미 노동부 JOLTs 7 월 채용공고 882 만건, 전월대비 33.8 만건 감소 및 예상치 950 만건 하회 ■ 전일 동향 - 전일 미국 채권금리는 미 경제지표 둔화로 하락. 미 노동부 JOLTs 7 월 채용공고는 2021 년 3 월 이후 최저수준인 882 만건으로 전월 대비 33.8 만건 감소, 예상치인 950 만건을  하회.  8 월  미국  소비자신뢰지수도  106.1 로  집계되며  전달  수정치인 114.0 및 예상치 116.0 을 하회 - 전일 국고채 금리는 하락. 전일 미국 채권시장 영향과 2024 년 예산안을 소화하며 강세. 기획재정부는 2024 년 국고채 발행 계획은 158.8 조원으로, 올해 167.8 조원 대비  9 조원  감소한  수준.  추가로  21 년만에  원화  외평채  발행  계획을  발표,  발행 한도는 18 조원으로 설정 ㅁ - 전일 크레딧 시장은 강보합. 개별기업 이슈로는 롯데케미칼이 회사채 수요예측에서 모집액의 5 배 규모를 기록, 가산금리는 2 년물 +5bp, 3 년물 +11bp로 결정 ■ 금일 전망 - 금일  채권시장은  강보합세 전망.  전일  미국 고용지표  둔화가  확인되며 금일 강세 재료로 반영될 것으로 예상. 다만, PCE 물가지표 및 비농업고용지수 등 주요 경제지표 발표를 앞두고 경계감은 유지 국내주요금리      해외주요금리  국고2YR 국고3YR 국고5YR 국고10YR 국고20YR 국고30YR 국고50YR 통안1YR 통안2YR CD91 일 회사채AA- 회사채BBB- 물가채 10YR 주 : 회사채 3Y 기준 수익률 1D 1M 12M US TN2YR 

In [197]:
df[df['증권사']=='교보증권'].iloc[4,2]

'Fixed Income Strategy 발간일자  : 2023/ 08/ 28   백윤민 연구위원 yoonmin1983@iprovest.com 확신이  없으면  모호할  수  밖에  없다   연준, 추가 금리인상에 목을 매는 것은 아니다 당사는 연준이 연내 기준금리 동결 기조를 유지할 것이라는 의견 유지. 일련의 불확실성 요인들을  감안하면  마지막까지  긴장의  끈을  놓을  수  없겠지만, 파월의장의  연설의  마지막  문장처럼  연준은  향후  금리인상  결정에  신중한  모습을  보일  것으로  예상.  연준이  추가  금리인상  가능성을  열어두고  있지만,  이전과  같이  인플레이션  제어를  위해  추가적인  금리인상이  시급한 상황은  아니라고  생각.  오히려  누적된  통화긴축으로  인한  파급효과를  점검하는  것이  필요한 시점으로 판단 금통위, 여전히 선제적 대응은 어렵다 당사는  한은이  어느  쪽이든  대내외  리스크  요인들이  현실화되기  전에  선제적으로  통화정책 대응에 나서기 어렵다는 점에서 연내 기준금리 동결 의견 유지. 한은이 경기적인 요소만 고려해 금리인하에 나서기도 어렵지만, 반대로 금리인상도 쉽지 않음. 이창용 총재는 국내 통화정책이  긴축적인  수준이라고  언급.  가계부채  증가,  연준의  추가  금리인상  가능성  등은  한은의 통화정책 운용에 부담 요인이. 그러나 이러한 불확실성 요인들이 실질적인 리스크로 연결되기 전에 한은이 선제적으로 통화정책 대응에 나설 명분은 제한적 투자전략: 불확실성 남아있지만, 여전히 매수 영역  글로벌  채권시장은  미국  국채발행  증가  부담과  연준  통화정책의  불확실성,  잭슨홀  심포지엄 경계감 등으로 약세. 국내 채권시장도 약세 흐름이 이어지는 가운데, 월 중 국고채 10년물 금리가 4%대로 상승하는 등 베어 스팁이 진행. 다만, 시장에 잔존하는 금리상승 재료들과 당분간 채권시장에 대한 경계감이 빠르게 완화되지 못할 수 있다는 점을 감안해도 현재 시장금리 레벨은 충

In [198]:
df[df['증권사']=='교보증권'].iloc[-3,2]

"  Daily 2017.11.01 Fixed Income 백윤민 02_3771_9175 / yoonmin1983@iprovest.com Market Comment 대내외 이벤트 불확실성 지속 상하   Today's Key Chart 미 미국 8 월 S&P/CS 주택가격지수, 예상치에 부합한 +5.92% YoY 기록 ■ 전일 동향  - 전일 국내 채권시장은 약세 마감.  국고채 30년물 입찰이 부진한 모습을 보이면서 초장기물 중심으로  금리가  급등.  국채  선물시장에서 외국인은  장단기물을  모두 순매도 -  해외:  미국  채권시장은  혼조  마감.    미국 주택  등  주요  경제지표들이  호조세를 보였지만, 미국 FOMC 에 대한 경계감과 트럼프 정치 불확실성 재료 등으로 금리 변동은 다소 제한되는 모습 ■ 금일 전망 - 금일  채권시장은  횡보 예상.   금리 상승에  따른 저가  매수 기대감은 유효하지만, 한은  기준금리  인상  경계감,  미국  FOMC,  차기  연준의장  인선  등  대내외 불확실성이 요인들이 이어지면서 투자심리가 뚜렷하게 개선되기 어려울 전망 S&P/CS 20개도시 주택지수Jan-00Jun-03Nov-06Apr-10Sep-13Feb-17국내주요금리    해외주요금리   수익률 1D 1M 12M US TN2YR US TN10YR US TB30YR DE 10YR UK 10YR FR 10YR IT  10YR ES 10YR JP 10YR CN 10YR AU 10YR 수익률 1D 1M 12M 통안1YR국고3YR국고10YR국고30YR통안2YR국고5YR국고20YR기준금리  국고채 Yield Curve  변동폭 1YR 1.5YR 2YR 2.5YR 3YR5YR 10YR 20YR 30YRJan-15Jul-15Jan-16Jul-16Jan-17Jul-17n 주요금리 스프레드    주요증시  / 환율 / Commodity  국고 30-국고10 국고20-국고10 국고10-국고5 국고10-국고3 국고5-국고3 국고3-기준 국고3-공사3  국고3-은

##### 교보증권 daily날릴까하는데 어떻게 생각하는지?

In [ ]:
#df.drop(df[(df['증권사']=='교보증권') & (df['텍스트'].str.startswith('Daily'))].index, inplace=True)

In [199]:
checklist.remove('교보증권')
checklist

[]

##### 저장

In [200]:
df.to_csv(f'/content/drive/Shareddrives/4조/data/bondreport/final/bond_report_preprocessed.csv')